In [1]:


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text

from tqdm.notebook import tqdm
import re

import gc

ModuleNotFoundError: ignored

In [ ]:
nadare_feature_dir = "../feature/FEAT26/"
nadare_util_dir = "../feature/common_utils/"
import os
os.makedirs(nadare_feature_dir, exist_ok=True)

In [ ]:
train_df = pd.read_csv("../input/foursquare-location-matching/train.csv", encoding="utf-8")
test_df = pd.read_csv("../input/foursquare-location-matching/test.csv", encoding="utf-8")
#test_df.loc[0, "name"] = np.NaN

In [ ]:
train_id_map = {v: i for i, v in enumerate(train_df["id"].values)}
train_df["ix"] = train_df["id"].map(train_id_map)
train_df["categories"] = train_df["categories"].fillna("nan")
train_df["pid"] = train_df["point_of_interest"].map({v: i for i, v in enumerate(train_df["point_of_interest"].unique())})

test_id_map = {v: i for i, v in enumerate(test_df["id"].values)}
test_df["ix"] = test_df["id"].map(test_id_map)
test_df["categories"] = test_df["categories"].fillna("nan")
test_df["pid"] = -1

In [ ]:
from sklearn.model_selection import KFold
group = -np.ones(len(train_df), dtype="int32")
g = 0
for g, (dev_p_ids, val_p_ids) in enumerate(KFold(n_splits=5, shuffle=True, random_state=42).split(train_df["pid"].unique())):
    ix = train_df["pid"].isin(val_p_ids)
    group[ix] = g
train_df["group"] = group
test_df["group"] = -1

In [ ]:
def get_value_with_default(dict_, value, default=0):
    return np.vectorize(lambda x: dict_.get(x, default))(value)

def make_category_dict(df, category_threthold = 1):
    cat_pairs = []
    all_category = set()
    for row in df["categories"].drop_duplicates().dropna().values:
        for cat in row.split(", "):
            if len(cat):
                cat_pairs.append([row, cat])
                all_category.add(cat)

    category_df = pd.DataFrame(cat_pairs, columns=["categories", "category"])
    categories_df = df[["ix", "categories"]].merge(category_df, on="categories", how="left").sort_values(by="ix")
    
    category_df = category_df.merge(categories_df.groupby("category")["ix"].count().reset_index().rename({"ix": "category_appear_count"}, axis=1),
                                    on="category",
                                    how="left")

    # if category doesn`t appear over threthold, it defines as out of vocab
    
    category_ix_df = category_df.drop_duplicates("category").sort_values(by="category_appear_count", ascending=False)[["category", "category_appear_count"]]
    category_ix_df["category_ix"] = np.where(category_ix_df["category_appear_count"] >= category_threthold, np.arange(len(category_ix_df)), -1)
    category_ix_df = category_ix_df.sort_values(by=["category_appear_count"], ascending=[False])
    category_ix_dict = pd.Series(index=category_ix_df["category"].values, data=category_ix_df["category_ix"].values).to_dict()
    #skip_nan_value
    return category_ix_dict

def get_category_ix(df, category_ix_dict):
    cat_pairs = []
    all_category = set()
    for row in df["categories"].drop_duplicates().dropna().values:
        for cat in row.split(", "):
            if len(cat):
                cat_pairs.append([row, cat])
                all_category.add(cat)

    category_df = pd.DataFrame(cat_pairs, columns=["categories", "category"])
    categories_df = df[["ix", "categories"]].merge(category_df, on="categories", how="left").sort_values(by="ix")

    categories_df["category_ix"] = get_value_with_default(category_ix_dict, categories_df["category"].values, -1)
    categories_df = categories_df[categories_df["category_ix"] >= 0]
    categories_ix = tf.RaggedTensor.from_value_rowids(values=tf.constant(categories_df["category_ix"].values, "int32"),
                                                              value_rowids=tf.constant(categories_df["ix"].values, "int32"),
                                                              nrows=len(df))
    return categories_ix

def remove_test_only_category(text):
    res = []
    for x in text.split(", "):
        if x in category_ix_dict.keys():
            res.append(x)
    return ", ".join(res)


category_ix_dict = make_category_dict(train_df, 1)
train_df["categories"] = np.vectorize(remove_test_only_category)(train_df["categories"])
train_categories_ix = get_category_ix(train_df, category_ix_dict)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
geocode_df = pd.read_csv(nadare_util_dir + "geocode.csv", header=None)
geocode_df.columns = ["latitude", "longitude", "country", "geo_city"]
geocode_df["geo_name"] = geocode_df["country"] + "_" + geocode_df["geo_city"]
geocode_df = geocode_df.groupby("geo_name")[["latitude", "longitude"]].mean().reset_index()
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(geocode_df[['latitude','longitude']]), geocode_df.index)
geo_index = knn.kneighbors(np.deg2rad(train_df[['latitude','longitude']]))[1].T[0]

In [ ]:
train_df["geo_name"] = geocode_df["geo_name"].values[geo_index]

In [ ]:
geo_name_vc = train_df["geo_name"].value_counts().head(253*8)
geo_name_index_df = geo_name_vc.reset_index()
geo_name_index_df.columns = ["geo_name", "geo_name_count"]
geo_name_index_df["geo_name_index"] = np.arange(len(geo_name_index_df))
geo_name_index_map = {geo_name: i for i, geo_name in enumerate(geo_name_index_df["geo_name"].values)}
geo_name_index_df = geo_name_index_df.merge(geocode_df[["geo_name", "latitude", "longitude"]], on="geo_name", how="left")
#geo_name_index_df = geo_name_index_df.merge(geo_name_position_df, on="geo_name", how="left")

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(geo_name_index_df[['latitude','longitude']]), geo_name_index_df["geo_name_index"])
pseudo_index = knn.kneighbors(np.deg2rad(train_df[['latitude','longitude']]))[1].T[0]
train_df["pseudo_geo_name"] = np.where(train_df["geo_name"].isin(geo_name_index_df["geo_name"].values), train_df["geo_name"].values, geo_name_index_df["geo_name"][pseudo_index].values)
train_df["pseudo_geo_name_ix"] = train_df["pseudo_geo_name"].map(geo_name_index_map)

In [ ]:
city_vc = train_df["city"].value_counts().head(253*8)
city_index_df = city_vc.reset_index()
city_index_df.columns = ["city", "city_count"]
city_index_df["city_index"] = np.arange(len(city_index_df))
city_index_map = {city: i for i, city in enumerate(city_index_df["city"].values)}
city_position_df = train_df.groupby("city")[["latitude", "longitude"]].mean().reset_index()
city_index_df = city_index_df.merge(city_position_df, on="city", how="left")

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(city_index_df[['latitude','longitude']]), city_index_df["city_index"])
pseudo_index = knn.kneighbors(np.deg2rad(train_df[['latitude','longitude']]))[1].T[0]
train_df["pseudo_city"] = np.where(train_df["city"].isin(city_index_df["city"].values), train_df["city"].values, city_index_df["city"][pseudo_index].values)
train_df["pseudo_city_ix"] = train_df["pseudo_city"].map(city_index_map)

In [ ]:
state_index_df = train_df["state"].value_counts().head(253*8).reset_index()
state_index_df.columns = ["state", "state_count"]
state_index_df["state_index"] = np.arange(len(state_index_df))
state_index_map = {state: i for i, state in enumerate(state_index_df["state"].values)}
state_position_df = train_df.groupby("state")[["latitude", "longitude"]].mean().reset_index()
state_index_df = state_index_df.merge(state_position_df, on="state", how="left")

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(state_index_df[['latitude','longitude']]), state_index_df["state_index"])
pseudo_index = knn.kneighbors(np.deg2rad(train_df[['latitude','longitude']]))[1].T[0]
train_df["pseudo_state"] = np.where(train_df["state"].isin(state_index_df["state"].values), train_df["state"].values, state_index_df["state"][pseudo_index].values)
train_df["pseudo_state_ix"] = train_df["pseudo_state"].map(state_index_map)

In [ ]:
train_df["country"] = train_df["country"].fillna("NAN")
country_index_df = train_df["country"].value_counts().reset_index()
country_index_df.columns = ["country", "country_count"]
country_index_df["country_index"] = np.arange(len(country_index_df))
country_index_map = {country: i for i, country in enumerate(country_index_df["country"].values, start=1)} # zero as nan
train_df["country_ix"] = train_df["country"].map(country_index_map)

In [ ]:
from fasttext import load_model
ft_model = load_model(nadare_util_dir + "lid.176.bin")
def predict_language(text):
    label, prob = ft_model.predict(text, 1)
    return list(zip([l.replace("__label__", "") for l in label], prob))[0][0]

train_df["name_language"] = np.vectorize(predict_language)(train_df["name"].fillna(""))
train_df["address_language"] = np.vectorize(predict_language)(train_df["address"].fillna(""))

def language_pattern(country, language):
    if country == "JP":
        return 0
    elif country == "TH":
        return 1
    elif country == "CN":
        return 2
    elif language == "ja":
        return 0
    elif language == "th":
        return 1
    elif language == "zh":
        return 2
    else:
        return 3
train_df["name_handle_pattern"] = np.vectorize(language_pattern)(train_df["country"], train_df["name_language"])     
train_df["address_handle_pattern"] = np.vectorize(language_pattern)(train_df["country"], train_df["address_language"])
lang_vc = train_df["name_language"].value_counts()
del ft_model

In [ ]:
from num2words import num2words
from collections import defaultdict

num2words_languages = ["ar", "cz", "en", "fr", "de", "fi", "es", "id", "kn", "ko", "kz", "lt", "lv", "pl", "ro", "ru", "sl",
                       "sr", "sv", "no", "dk", "pt", "he", "it", "vi", "th", "tr", "nl", "uk", "te", "hu"]

word_number_df = []
for lang in tqdm(num2words_languages):
    for i in range(1001):
        try:
            word = num2words(i, lang=lang)
        except:
            word = np.NaN
        try:
            ordinal = num2words(i, lang=lang, to="ordinal")
        except:
            ordinal = np.NaN
        try:
            ordinal_num = num2words(i, lang=lang, to="ordinal_num")
        except:
            ordinal_num = np.NaN
        if (pd.isna(ordinal)) or (pd.isna(ordinal_num)) or (ordinal == ordinal_num):
            ordinal_num = i
        if lang == "ko" and not pd.isna(ordinal):
            ordinal = ordinal[:-3]
            ordinal_num = ordinal_num[:-3] 
        
        word_number_df.append({"lang": lang,
                               "number": i,
                               "word": word,
                               "ordinal": ordinal,
                               "ordinal_num": ordinal_num})

word_number_df = pd.DataFrame(word_number_df)
word_number_df["lang_count"] = word_number_df["lang"].apply(lambda x: lang_vc.get(x, 0))

word2num_dict = {}
possibly_have_next_dict = {}

for i, row in word_number_df.sort_values(by="lang_count").iterrows():
    if not pd.isna(row["word"]):
        word = row["word"]
        if not word in possibly_have_next_dict.keys():
            possibly_have_next_dict[word] = False

        word2num_dict[word] = str(row["number"])
        word2num_dict[word.replace("-", " ")] = str(row["number"])
        
        wsp = word.split()
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        wsp = word.replace("-", " ").split()[:-1]
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        
    if (not pd.isna(row["ordinal"])) and (not pd.isna(row["ordinal_num"])):
        word = row["ordinal"]
        if not word in possibly_have_next_dict.keys():
            possibly_have_next_dict[word] = False
        word2num_dict[word] = str(row["ordinal_num"])
        word2num_dict[word.replace("-", " ")] = str(row["ordinal_num"])
        
        wsp = word.split()
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        wsp = word.replace("-", " ").split()[:-1]
        for i in range(1, len(wsp)-1):
            possibly_have_next_dict[" ".join(wsp[:i])] = True
        

  0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
import string
import pycnnum
import pythainlp
from pythainlp.util import thai_digit_to_arabic_digit
from sudachipy import Dictionary, SplitMode
import mojimoji
import pykakasi
from unidecode import unidecode
import emoji

emoji_set = set()
for k, v in emoji.UNICODE_EMOJI.items():
    emoji_set.update(v.keys())
thai_word2num_dict = {num2words(i, lang="th"):str(i) for i in range(10001)}
CHINESE_DIGITS_PATTERN = re.compile(r'[〇〇零一二三四五六七八九零壹贰叁肆伍陆柒捌玖十百千万拾佰仟萬]+')
remove_punctuations = '!"#$%\'()*+,./:;<=>?@[\\]^_`{|}~'
punctuation_translater = str.maketrans(remove_punctuations, " "*len(remove_punctuations))
japanese_char2num_translater = str.maketrans("〇零一壱二弐三参四肆五伍六陸七漆八捌九玖", "00112233445566778899")


kakasi = pykakasi.kakasi()
kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
conversion = kakasi.getConverter()

force_katakana = pykakasi.kakasi()
kakasi.setMode('H', 'K')  # Convert Hiragana into alphabet
kakasi.setMode('K', 'K')  # Convert Katakana into alphabet
force_katakana.setMode('J', 'K')  # Convert Hiragana into alphabet
force_katakana_conversion = force_katakana.getConverter()

sudachi_tokenizer = Dictionary(dict_type="full").create()
sudachi_tokenize_mode = SplitMode.A

zh_segmenter = tf_text.HubModuleTokenizer(nadare_util_dir + "zh_segmentation")

def remove_emoji(x):
    return ''.join([c for c in x if c not in emoji_set])

def remove_marks(x):
    x = re.sub(r'[^a-zA-Z0-9& ]', ' ', x)
    x = re.sub(r' +', ' ', x)
    return x

def zenhan_normalize(x):
    return mojimoji.han_to_zen(mojimoji.zen_to_han(x, kana=False), digit=False, ascii=False)

def japanese_wakachi_reading(morphemes):
    readings = []
    for m in morphemes:
        if (m.normalized_form() == " ") or (m.reading_form() == "キゴウ"):
            continue
        if m.normalized_form().endswith("丁目") and m.normalized_form() != "丁目":
            norm = m.normalized_form().translate(japanese_char2num_translater)
            norm = re.sub(r"(\d+)", r" \1 ", "6丁目")
            rs = japanese_wakachi_reading(sudachi_tokenizer.tokenize(norm, sudachi_tokenize_mode))
            readings.extend(rs)
        elif m.normalized_form().isdigit():
            readings.append(m.normalized_form())
        else:
            readings.append(force_katakana_conversion.do(m.reading_form()))
    return readings

def handle_japanese(x):
    x = zenhan_normalize(x)
    morphemes = sudachi_tokenizer.tokenize(x, sudachi_tokenize_mode)
    reading_forms = japanese_wakachi_reading(morphemes)
    romanize_forms = conversion.do(" ".join(reading_forms))
    return " ".join(reading_forms), romanize_forms

def decode_list(x):
    if type(x) is list:
        return list(map(decode_list, x))
    return x.decode("UTF-8")

def decode_utf8_tensor(x):
    return list(map(decode_list, x.to_list()))

def handle_chinese(x):
    x = x.replace("'", "").translate(punctuation_translater).lower()
    if re.fullmatch("\s*", x):
        return "", ""
    with tf.device("CPU:0"):
        words_list = decode_utf8_tensor(zh_segmenter.tokenize(x.split()))
        
    number_words = []
    romanized_words = []
    for words in words_list:
        num_word = []
        roman_word = []
        for word in words:
            if re.fullmatch(CHINESE_DIGITS_PATTERN, word):
                word = str(pycnnum.cn2num(word))
            if (word[0]) == "第" and re.fullmatch(CHINESE_DIGITS_PATTERN, word[1:]):
                num = str(pycnnum.cn2num(word[1:]))
                word = "第"
                num_word.append(word + num)
                roman_word.append(unidecode(word[0]).lower().replace(" ", "") + " " + num)
            else:
                num_word.append(word)
                roman_word.append(unidecode(word).lower().replace(" ", ""))
        number_words.append("".join(num_word))
        romanized_words.append(" ".join(roman_word))
    return " ".join(number_words), " ".join(romanized_words)

def handle_thai(x):
    x = thai_digit_to_arabic_digit(x)
    tokenized_word = pythainlp.tokenize.word_tokenize(x, engine="newmm")
    number_words = []
    romanized_words = []
    for word in tokenized_word:
        word = thai_word2num_dict.get(word, word)
        number_words.append(word)
        try:
            romanized_words.append(pythainlp.romanize(word, engine='royin'))
        except:
            pass
    return "".join(number_words), " ".join(romanized_words)


def language_normalize(word):
    result = []
    word = word.replace("'", "").translate(punctuation_translater).lower()
    tmp = ""
    for w in word.split():
        if len(tmp):
            tmp = tmp + " " + w
            if tmp in possibly_have_next_dict.keys():
                if possibly_have_next_dict[tmp]:
                    continue
                else:
                    result.append(word2num_dict[tmp])
                    tmp = ""
        else:
            if w in possibly_have_next_dict.keys():
                if possibly_have_next_dict[w]:
                    tmp = w
                    continue
                else:
                    result.append(word2num_dict[w])
            else:
                result.append(w)
    if len(tmp):
        result.append(word2num_dict.get(tmp, tmp))
            
    return re.sub(r' +', ' ', " ".join(result))

def language_handle(handle_pattern, text):
    if handle_pattern == 0:
        normalized, romanized = handle_japanese(text)
    elif handle_pattern == 1:
        normalized, romanized = handle_thai(text)
    elif handle_pattern == 2:
        normalized, romanized = handle_chinese(text)
    return normalized, romanized


C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:22: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:23: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:24: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:25: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  conversion = kakasi.getConverter()
C:\Users\n

In [ ]:
name_handle_index = np.where((train_df["name_handle_pattern"] <= 2) & (~train_df["name"].isna()))[0]

name_normalized = []
name_romanized = []
for pattern, name in tqdm(zip(train_df["name_handle_pattern"].values[name_handle_index],
                              train_df["name"].fillna("").astype(str).values[name_handle_index]), total=len(name_handle_index)):
    normalized, romanized = language_handle(pattern, name)
    name_normalized.append(normalized)
    name_romanized.append(romanized)

  0%|          | 0/150443 [00:00<?, ?it/s]

C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:62: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  readings.append(force_katakana_conversion.do(m.reading_form()))
C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:69: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  romanize_forms = conversion.do(" ".join(reading_forms))


In [ ]:
address_handle_index = np.where((train_df["address_handle_pattern"] <= 2) & (~train_df["address"].isna()))[0]

address_normalized = []
address_romanized = []
for pattern, address in tqdm(zip(train_df["address_handle_pattern"].values[address_handle_index],
                              train_df["address"].fillna("").astype(str).values[address_handle_index]), total=len(address_handle_index)):
    normalized, romanized = language_handle(pattern, address)
    address_normalized.append(normalized)
    address_romanized.append(romanized)

  0%|          | 0/90478 [00:00<?, ?it/s]

C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:62: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  readings.append(force_katakana_conversion.do(m.reading_form()))
C:\Users\nadare\AppData\Local\Temp/ipykernel_32624/2681526445.py:69: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  romanize_forms = conversion.do(" ".join(reading_forms))


In [ ]:
import gc
del zh_segmenter, kakasi, conversion, sudachi_tokenizer
gc.collect()

10576

In [ ]:
train_df["name_normalized"] = np.vectorize(language_normalize)(train_df["name"].fillna("").astype(str))
train_df["address_normalized"] = np.vectorize(language_normalize)(train_df["address"].fillna("").astype(str))


from_cols = ["name_normalized", "address_normalized"]
to_cols = ['sp_name', "sp_address_raw"]
for from_, to_ in zip(from_cols, to_cols):
    train_df[to_] = train_df[from_].fillna('').apply(unidecode)
    train_df[to_] = train_df[to_].str.lower()
    train_df[to_] = train_df[to_].apply(remove_emoji)
    train_df[to_] = train_df[to_].apply(remove_marks)
    
train_df.loc[name_handle_index, "name_normalized"] = name_normalized
train_df.loc[name_handle_index, "sp_name"] = name_romanized

train_df.loc[address_handle_index, "address_normalized"] = address_normalized
train_df.loc[address_handle_index, "sp_address_raw"] = address_romanized

In [ ]:
import sentencepiece as spm

train_df[["sp_name"]][~np.vectorize(lambda x: bool(re.fullmatch("\s*", x)))(train_df["sp_name"].fillna(""))].to_csv("./texts/sp_name.txt", index=None, encoding="utf-8", header=None)
train_df[["sp_address_raw"]][~np.vectorize(lambda x: bool(re.fullmatch("\s*", x)))(train_df["sp_address_raw"].fillna(""))].to_csv("./texts/sp_address.txt", index=None, encoding="utf-8", header=None)

if True:
    spm.SentencePieceTrainer.train(
        input="./texts/sp_name.txt",
        model_type="unigram",
        split_by_whitespace=True,
        model_prefix=nadare_feature_dir + "sp_name",
        character_coverage=1.,
        vocab_size=32000,
    )

if True:
    spm.SentencePieceTrainer.train(
        input="./texts/sp_address.txt",
        model_type="unigram",
        split_by_whitespace=True,
        model_prefix=nadare_feature_dir + "sp_address",
        character_coverage=1.,
        vocab_size=32000,
    )



In [ ]:
from sklearn.neighbors import KNeighborsRegressor

address_index = np.where(~np.vectorize(lambda x: bool(re.fullmatch("\s*", x)))(train_df["sp_address_raw"].fillna("")))[0]
knn = KNeighborsRegressor(n_neighbors=3, 
                          metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(train_df[['latitude','longitude']].values)[address_index], address_index)
address_neiighbor = address_index[knn.kneighbors(np.deg2rad(train_df[['latitude','longitude']].values))[1]]
train_df["sp_address"] = np.vectorize(lambda x: " ".join(x))(pd.Series(train_df["sp_address_raw"].values[address_neiighbor].tolist()))

In [ ]:
train_df["sp_name"] = train_df["sp_name"].apply(remove_marks)
train_df["sp_address"] = train_df["sp_address"].apply(remove_marks)

from tensorflow_text import BertTokenizer, SentencepieceTokenizer
preprocessor = hub.KerasLayer(
    "../input/universalsentenceencodercmlmbr/multilingual-base-br")
encoder = hub.KerasLayer(
    "../input/universalsentenceencodercmlmbr/multilingual-preprocess")

def trimer(X):
    trim_size = tf.reduce_max(tf.reduce_sum(X["input_mask"], axis=1))
    Y = {"input_mask": tf.slice(X["input_mask"], [0, 0], [-1, trim_size]),
         "input_type_ids": tf.slice(X["input_type_ids"], [0, 0], [-1, trim_size]),
         "input_word_ids": tf.slice(X["input_word_ids"], [0, 0], [-1, trim_size]),
         }
    return Y

batch_size = 128

In [ ]:
# あまり効かない
"""
batch_size = 128

token_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["name"].fillna("").values))\
            .batch(batch_size)\
            .map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE)\
            .map(trimer, num_parallel_calls=tf.data.AUTOTUNE)\
            .prefetch(tf.data.AUTOTUNE)

embeddings = []
for text in tqdm(token_ds, total=len(train_df)//batch_size):
    embeddings.append(encoder(text)['pooled_output'])

name_embeddings = tf.concat(embeddings, axis=0)

from sklearn.decomposition import PCA
name_embeddings = tf.convert_to_tensor(PCA(n_components=128).fit_transform(name_embeddings.numpy()))

model = open('./sentencepiece/sp_name.model', "rb").read()
sp_tokenizer = SentencepieceTokenizer(model)

token_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["sp_name"]))\
            .batch(batch_size)\
            .map(sp_tokenizer.tokenize, num_parallel_calls=tf.data.AUTOTUNE)\
            .prefetch(tf.data.AUTOTUNE)
name_init_embedding = tf.zeros([32000, 128])
name_init_count = tf.zeros([32000])

for token in tqdm(token_ds):
    name_init_embedding = tf.tensor_scatter_nd_add(name_init_embedding,
                                                   tf.expand_dims(token.values, axis=1),
                                                   tf.gather(name_embeddings, token.value_rowids()))
    name_init_count = tf.tensor_scatter_nd_add(name_init_count,
                                               tf.expand_dims(token.values, axis=1),
                                               tf.ones(tf.shape(token.value_rowids())))    
name_init_embedding = tf.nn.l2_normalize(name_init_embedding / (tf.expand_dims(name_init_count, axis=1) + tf.keras.backend.epsilon()), axis=1)
pd.to_pickle(name_init_embedding.numpy(), "../processed/name_init_embedding.pickle")
"""

'\nbatch_size = 128\n\ntoken_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["name"].fillna("").values))            .batch(batch_size)            .map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE)            .map(trimer, num_parallel_calls=tf.data.AUTOTUNE)            .prefetch(tf.data.AUTOTUNE)\n\nembeddings = []\nfor text in tqdm(token_ds, total=len(train_df)//batch_size):\n    embeddings.append(encoder(text)[\'pooled_output\'])\n\nname_embeddings = tf.concat(embeddings, axis=0)\n\nfrom sklearn.decomposition import PCA\nname_embeddings = tf.convert_to_tensor(PCA(n_components=128).fit_transform(name_embeddings.numpy()))\n\nmodel = open(\'./sentencepiece/sp_name.model\', "rb").read()\nsp_tokenizer = SentencepieceTokenizer(model)\n\ntoken_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["sp_name"]))            .batch(batch_size)            .map(sp_tokenizer.tokenize, num_parallel_calls=tf.data.AUTOTUNE)            .prefetch(tf.data.AUTOTUNE)\nname_init_embe

In [ ]:
# あまり効かない
"""
from tensorflow_text import BertTokenizer, SentencepieceTokenizer
batch_size = 128

token_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["address"].fillna("").values))\
            .batch(batch_size)\
            .map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE)\
            .map(trimer, num_parallel_calls=tf.data.AUTOTUNE)\
            .prefetch(tf.data.AUTOTUNE)

embeddings = []
for text in tqdm(token_ds, total=len(train_df)//batch_size):
    embeddings.append(encoder(text)['pooled_output'])

address_embeddings = tf.concat(embeddings, axis=0)

from sklearn.decomposition import PCA
address_embeddings = tf.convert_to_tensor(PCA(n_components=128).fit_transform(address_embeddings.numpy()))

model = open('./sentencepiece/sp_address.model', "rb").read()
sp_tokenizer = SentencepieceTokenizer(model)

token_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["sp_address"].fillna("").values))\
            .batch(batch_size)\
            .map(sp_tokenizer.tokenize, num_parallel_calls=tf.data.AUTOTUNE)\
            .prefetch(tf.data.AUTOTUNE)
address_init_embedding = tf.zeros([32000, 128])
address_init_count = tf.zeros([32000])

for token in tqdm(token_ds):
    address_init_embedding = tf.tensor_scatter_nd_add(address_init_embedding,
                                                   tf.expand_dims(token.values, axis=1),
                                                   tf.gather(address_embeddings, token.value_rowids()))
    address_init_count = tf.tensor_scatter_nd_add(address_init_count,
                                               tf.expand_dims(token.values, axis=1),
                                               tf.ones(tf.shape(token.value_rowids())))    
address_init_embedding = tf.nn.l2_normalize(address_init_embedding / (tf.expand_dims(address_init_count, axis=1) + tf.keras.backend.epsilon()), axis=1)
pd.to_pickle(address_init_embedding.numpy(), "../processed/address_init_embedding.pickle")
"""

'\nfrom tensorflow_text import BertTokenizer, SentencepieceTokenizer\nbatch_size = 128\n\ntoken_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["address"].fillna("").values))            .batch(batch_size)            .map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE)            .map(trimer, num_parallel_calls=tf.data.AUTOTUNE)            .prefetch(tf.data.AUTOTUNE)\n\nembeddings = []\nfor text in tqdm(token_ds, total=len(train_df)//batch_size):\n    embeddings.append(encoder(text)[\'pooled_output\'])\n\naddress_embeddings = tf.concat(embeddings, axis=0)\n\nfrom sklearn.decomposition import PCA\naddress_embeddings = tf.convert_to_tensor(PCA(n_components=128).fit_transform(address_embeddings.numpy()))\n\nmodel = open(\'./sentencepiece/sp_address.model\', "rb").read()\nsp_tokenizer = SentencepieceTokenizer(model)\n\ntoken_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_df["sp_address"].fillna("").values))            .batch(batch_size)            .map(sp_tokenize

In [ ]:
"""
batch_size = 128

token_ds = tf.data.Dataset.from_tensor_slices(tf.constant(pd.Series(category_ix_dict).sort_values().index.values))\
            .batch(batch_size)\
            .map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE)\
            .map(trimer, num_parallel_calls=tf.data.AUTOTUNE)\
            .prefetch(tf.data.AUTOTUNE)

embeddings = []
for text in tqdm(token_ds, total=len(category_ix_dict)//batch_size):
    embeddings.append(encoder(text)['pooled_output'])

category_init_embedding = tf.concat(embeddings, axis=0)
category_init_embedding = tf.nn.l2_normalize(tf.convert_to_tensor(PCA(n_components=128).fit_transform(category_init_embedding.numpy())), axis=1)
pd.to_pickle(category_init_embedding.numpy(), "../processed/category_init_embedding.pickle")"""

'\nbatch_size = 128\n\ntoken_ds = tf.data.Dataset.from_tensor_slices(tf.constant(pd.Series(category_ix_dict).sort_values().index.values))            .batch(batch_size)            .map(preprocessor, num_parallel_calls=tf.data.AUTOTUNE)            .map(trimer, num_parallel_calls=tf.data.AUTOTUNE)            .prefetch(tf.data.AUTOTUNE)\n\nembeddings = []\nfor text in tqdm(token_ds, total=len(category_ix_dict)//batch_size):\n    embeddings.append(encoder(text)[\'pooled_output\'])\n\ncategory_init_embedding = tf.concat(embeddings, axis=0)\ncategory_init_embedding = tf.nn.l2_normalize(tf.convert_to_tensor(PCA(n_components=128).fit_transform(category_init_embedding.numpy())), axis=1)\npd.to_pickle(category_init_embedding.numpy(), "../processed/category_init_embedding.pickle")'

knn = KNeighborsRegressor(n_neighbors=128 + train_df["pid"].value_counts().max(), 
                              metric='haversine', 
                          n_jobs=-1)
knn.fit(np.deg2rad(train_df[['latitude','longitude']]), train_df["ix"])
neighbors_dist, neighbors_ix = knn.kneighbors(np.deg2rad(train_df[['latitude','longitude']]), return_distance=True)
neighbors_dist = np.where(train_df["pid"].values.reshape(-1, 1) != train_df["pid"].values[neighbors_ix], neighbors_dist, 1e9).astype(np.float32)
order = np.argsort(neighbors_dist, axis=1)
neighbors_dist = np.take_along_axis(neighbors_dist, order, axis=1)[:, :128]
neighbors_ix = np.take_along_axis(neighbors_ix, order, axis=1)[:, :128]
pd.to_pickle((neighbors_dist, neighbors_ix), "../processed/neighbor_pairs.pickle")

In [ ]:
name_init_embedding = None#tf.convert_to_tensor(pd.read_pickle("../processed/name_init_embedding.pickle"))
address_init_embedding = None#tf.convert_to_tensor(pd.read_pickle("../processed/address_init_embedding.pickle"))
#category_init_embedding = tf.convert_to_tensor(pd.read_pickle("../processed/category_init_embedding.pickle"))
neighbor_dist, neighbor_ix = pd.read_pickle("../processed/neighbor_pairs.pickle")
neighbor_dist = -np.log(neighbor_dist + tf.keras.backend.epsilon())

neighbor_ix = tf.convert_to_tensor(neighbor_ix.astype(np.int32))
neighbor_dist = tf.convert_to_tensor(neighbor_dist.astype(np.float32))


In [ ]:
ix_params_dict = {"country": {"dimention": 64, "num_category": len(country_index_df)+1},
                  "state": {"dimention": 64, "num_category": len(state_index_df)},
                  "city": {"dimention": 64, "num_category": len(city_index_df)},
                  "geo_name": {"dimention": 64, "num_category": len(geo_name_index_df)}
                  }
train_ix_values = {"country": tf.convert_to_tensor(train_df["country_ix"].values.astype(np.int32)),
                   "state": tf.convert_to_tensor(train_df["pseudo_state_ix"].values.astype(np.int32)),
                   "city": tf.convert_to_tensor(train_df["pseudo_city_ix"].values.astype(np.int32)),
                   "geo_name": tf.convert_to_tensor(train_df["pseudo_geo_name_ix"].values.astype(np.int32)),
                   }

class IxEmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self, ix_params_dict, num_layer=1, out_dim=128):
        super(IxEmbeddingLayer, self).__init__()
        self.ix_params_dict = ix_params_dict
        self.num_layer = num_layer
        self.out_dim = out_dim
        self.embedding_layers = {k: tf.keras.layers.Embedding(v["num_category"], v["dimention"]) for k, v in ix_params_dict.items()}
        self.denses = [tf.keras.layers.Dense(out_dim, activation="gelu")]
        self.out = tf.keras.layers.Dense(out_dim)
        self.drop_out = tf.keras.layers.Dropout(0.1)
    
    def call(self, ix_values):
        results = []
        for k, v in ix_values.items():
            results.append(self.embedding_layers[k](v))
        X = tf.concat(results, axis=-1)
        
        for i in range(self.num_layer):
            X = self.drop_out(self.denses[i](X))
        return self.out(X)
        

In [ ]:
def approx_ranks(logits):
    r"""Computes approximate ranks given a list of logits.
    Given a list of logits, the rank of an item in the list is one plus the total
    number of items with a larger logit. In other words,
    rank_i = 1 + \sum_{j \neq i} I_{s_j > s_i},
    where "I" is the indicator function. The indicator function can be
    approximated by a generalized sigmoid:
    I_{s_j < s_i} \approx 1/(1 + exp(-(s_j - s_i)/temperature)).
    This function approximates the rank of an item using this sigmoid
    approximation to the indicator function. This technique is at the core
    of "A general approximation framework for direct optimization of
    information retrieval measures" by Qin et al.
    Args:
    logits: A `Tensor` with shape [batch_size, list_size]. Each value is the
      ranking score of the corresponding item.
    Returns:
    A `Tensor` of ranks with the same shape as logits.
    """
    list_size = tf.shape(input=logits)[1]
    x = tf.tile(tf.expand_dims(logits, 2), [1, 1, list_size])
    y = tf.tile(tf.expand_dims(logits, 1), [1, list_size, 1])
    pairs = tf.sigmoid(y - x)
    return tf.reduce_sum(input_tensor=pairs, axis=-1) + .5
    
    
class RankMSE(tf.keras.layers.Layer):
    def __init__(self):
        super(RankMSE, self).__init__()
    
    def call(self, y_true, y_pred):        
        weight = 1 / y_true
        rmse = tf.reduce_sum(tf.math.sqrt(tf.math.square(y_true - y_pred))*weight, axis=-1) / tf.reduce_sum(weight, axis=1)
        
        return tf.reduce_mean(rmse)

In [ ]:
class MixerModel(tf.keras.Model):
    
    def __init__(self, name_model, address_model, ix_emb_layer, cat_emb_layer, num_layer=2, num_dim=128):
        super(MixerModel, self).__init__()
        self.num_layer = num_layer
        self.num_dim = num_dim
        self.name_model = name_model
        self.address_model = address_model
        self.ix_emb_layer = ix_emb_layer
        self.cat_emb_layer = cat_emb_layer
        
        self.tokenize_norm = [tf.keras.layers.LayerNormalization() for i in range(4)]
    
        self.drop_out = tf.keras.layers.Dropout(0.1)
        
        self.filter_denses = [tf.keras.layers.DepthwiseConv2D(kernel_size=(1, 4),
                                                              strides=1,
                                                              padding="valid",
                                                              depth_multiplier=4,
                                                              activation="gelu") for _ in range(self.num_layer)]
        self.out_denses = [tf.keras.layers.Dense(num_dim) for _ in range(self.num_layer)]
        
        #self.pointwise_filter_denses = [tf.keras.layers.Dense(num_dim, activation="gelu") for _ in range(self.num_layer)]
        #self.pointwise_out_denses = [tf.keras.layers.Dense(num_dim) for _ in range(self.num_layer)]
        self.norm_layers = [tf.keras.layers.LayerNormalization() for _ in range(self.num_layer)]
        
    def check_tensor(self, x, ix):
        if isinstance(x, tf.RaggedTensor):
            x = x.to_tensor()
        return x
        
    def call(self, info, training=False):
        ix, name, address, position, ix_values, categories, pids, size = info
        X_name = self.tokenize_norm[0](tf.reduce_sum(self.name_model(name), axis=-2))
        X_address = self.tokenize_norm[1](tf.reduce_sum(self.address_model(address), axis=-2))
        X_ix = self.tokenize_norm[2](self.drop_out(self.ix_emb_layer(ix_values)))
        X_categories = self.tokenize_norm[3](tf.reduce_sum(self.drop_out(self.cat_emb_layer(categories)), axis=-2))
        
        X = tf.stack([X_name, X_address, X_ix, X_categories], axis=-2)
        original_shape = tf.shape(X)
        X = tf.expand_dims(X, axis=1)
        expand_shape = tf.shape(X)
        for i in range(self.num_layer):
            X_ = self.norm_layers[i](X)
            X_ = self.out_denses[i](tf.reshape(self.filter_denses[i](X_), expand_shape))
            X = X + X_
        X = tf.reshape(X, original_shape)
        return tf.reduce_mean(X, axis=-2)

def simcse_task(mixer_model, loss_func, neighbor_info):
    X_a = tf.nn.l2_normalize(mixer_model(neighbor_info, training=True), axis=-1)
    X_b = tf.nn.l2_normalize(mixer_model(neighbor_info, training=True), axis=-1)
    
    X_p = tf.reshape(neighbor_info[-2], [-1, neighbor_info[-1]])
    
    N = tf.shape(X_p)[0]
    R = tf.shape(X_p)[1]
    
    shape = tf.shape(X_a)
    X_a = tf.reshape(X_a, [-1, neighbor_info[-1], shape[-1]])
    X_b = tf.reshape(X_b, [-1, neighbor_info[-1], shape[-1]])
    shape = tf.shape(X_a)
    
    cossim_c = tf.reshape(tf.einsum("nad,nbd->nab", X_a, X_b), [shape[0]*shape[1], shape[1]])
    cossim_r = tf.reshape(tf.einsum("nad,nbd->nab", tf.transpose(X_a, [1, 0, 2]), tf.transpose(X_b, [1, 0, 2])), [shape[0]*shape[1], shape[0]])
 
    mask_c = tf.reshape(tf.expand_dims(X_p, axis=1) != tf.expand_dims(X_p, axis=2), [shape[0]*shape[1], shape[1]])
    mask_r = tf.reshape(tf.expand_dims(X_p, axis=0) != tf.expand_dims(X_p, axis=1), [shape[0]*shape[1], shape[0]])    

    label_c = tf.reshape(tf.repeat(tf.expand_dims(tf.eye(tf.shape(X_a)[1]), axis=0), tf.shape(X_a)[0], axis=0), tf.shape(cossim_c))
    label_r = tf.reshape(tf.repeat(tf.expand_dims(tf.eye(tf.shape(X_a)[0]), axis=1), tf.shape(X_a)[1], axis=0), tf.shape(cossim_r))
    
    mask_c = tf.math.logical_or(label_c == 1., mask_c)
    mask_r = tf.math.logical_or(label_r == 1., mask_r)
    
    cossim_c = tf.where(mask_c, cossim_c, -1.)
    cossim_r = tf.where(mask_r, cossim_r, -1.)
    
    scale_c = tf.sqrt(2.) * tf.math.log(tf.cast(tf.shape(label_c)[1], "float32") - 1.)
    scale_r = tf.sqrt(2.) * tf.math.log(tf.cast(tf.shape(label_r)[1], "float32") - 1.)
    loss = loss_func(label_c, cossim_c * scale_c) + loss_func(label_r, cossim_r * scale_r)
    return loss


In [ ]:
class DimFreeLayerNormalization(tf.keras.layers.Layer):
    def __init__(self, name):
        super(DimFreeLayerNormalization, self).__init__()
        self.g = tf.Variable(1., dtype="float32", trainable=True, name="/" + name + "_a")
        self.a = tf.Variable(0., dtype="float32", trainable=True, name="/" + name + "_b")
        
    def call(self, X):
        X_mean = tf.reduce_mean(X, axis=-1, keepdims=True)
        X_std = tf.math.reduce_std(X, axis=-1, keepdims=True)
        X = self.g * (X-X_mean) / (X_std + tf.keras.backend.epsilon()) + self.a
        return X       

In [ ]:
class CategoryEmbeddingLayer(tf.keras.layers.Layer):
    
    def __init__(self, num_categories, dim_categories, init_embedding=None):
        super(CategoryEmbeddingLayer, self).__init__()
        self.num_categories = num_categories
        self.dim_categories = dim_categories
        self.category_dense = tf.Variable(tf.keras.initializers.GlorotUniform()(shape=(num_categories, dim_categories)), trainable=True, name=self.name + "/category_embedding")
        if not init_embedding is None:
            self.category_dense.assign(init_embedding)
            
    def call(self, C):
        X = tf.gather(self.category_dense, C)
        return X

In [ ]:
import tensorflow as tf
from tensorflow_text import SentencepieceTokenizer

class SupervisedContrastiveLoss(tf.keras.layers.Layer):
    def __init__(self, from_logits=False):
        super(SupervisedContrastiveLoss, self).__init__()
        if not from_logits:
            NotImplementedError
        
    @tf.function
    def call(self, true, pred, sample_weight=None):
        if sample_weight is None:
            sample_weight = tf.ones(tf.shape(pred), dtype=tf.float32)
        epred = tf.exp(pred) * sample_weight
        scale = tf.reduce_sum(epred, axis=1, keepdims=True)
        loss = tf.reduce_sum((tf.math.log(scale - epred * true) - true*pred) * true * sample_weight) / tf.reduce_sum(true*sample_weight)
        return loss
    
class SentencePieceEmbeddingLayer(tf.keras.layers.Layer):
    
    def __init__(self, vocab_size, out_dim, sp_model_path, init_embedding=None):
        super(SentencePieceEmbeddingLayer, self).__init__()
        self.vocab_size = vocab_size
        self.out_dim = out_dim
        model = open(sp_model_path, "rb").read()
        self.tokenizer = SentencepieceTokenizer(model)
        self.embedding = tf.keras.layers.Embedding(vocab_size, out_dim)
        if not init_embedding is None:
            self.embedding(0)
            self.embedding.trainable_variables[0].assign(init_embedding)
        self.drop_out = tf.keras.layers.Dropout(0.1)
    
    def call(self, X):
        token = self.tokenizer.tokenize(X)
        X = self.drop_out(self.embedding(token))
        return X
    
class DpConvWrapper(tf.keras.layers.Layer):
    
    def __init__(self, sp_layer, num_layer=2):
        super(DpConvWrapper, self).__init__()
        self.sp_layer = sp_layer
        self.num_layer = num_layer
        self.out_dim = self.sp_layer.out_dim
        self.norms = [tf.keras.layers.LayerNormalization() for i in range(num_layer)]
        self.dep_convs = [tf.keras.layers.DepthwiseConv1D(kernel_size=3, strides=1, padding="same", activation="gelu") for _ in range(num_layer)]
        self.denses = [tf.keras.layers.Dense(sp_layer.out_dim) for _ in range(num_layer)]
        self.drop_out = tf.keras.layers.Dropout(0.1)
        
        
    def call(self, X):
        token = name_dpc.sp_layer.tokenizer.tokenize(X) + 1
        start = tf.cast(tf.concat([tf.zeros([1], dtype="int64"), tf.cumsum(token.row_lengths())], axis=0), "int32")
        row_ids = tf.cast(token.value_rowids(), "int32")
        col_ids = tf.range(tf.shape(token.value_rowids())[0], dtype="int32") - tf.gather(start, token.value_rowids())
        X = self.drop_out(name_dpc.sp_layer.embedding(token.to_tensor(default_value=2)))
        rank = tf.rank(X)
        shape = tf.shape(X)
        
        for i in range(name_dpc.num_layer):
            X_ = name_dpc.norms[i](X)
            X_ = name_dpc.dep_convs[i](X_)
            X_ = name_dpc.denses[i](X_)
            X += X_
            
        X = tf.RaggedTensor.from_value_rowids(tf.gather_nd(X, tf.stack([row_ids, col_ids], axis=1)),
                                              token.value_rowids(),
                                              nrows=tf.cast(shape[0], "int64"))
        return X        

    
class SkipgramModel(tf.keras.Model):
    
    def __init__(self, name_spe):
        super(SkipgramModel, self).__init__()
        self.name_spe = name_spe
        
    #@tf.function(experimental_relax_shapes=True)
    def call(self, name, n_name, n_pid):
        X = skipgram_model.name_spe(name)
        X_sum_neighbor = tf.reduce_sum(skipgram_model.name_spe(tf.slice(n_name, [0, 1], [-1, -1])), axis=2).to_tensor()
        name_skip_loss = skipgram_model.skipgram_task(X, X_sum_neighbor, n_pid)

        return name_skip_loss
    
    #@tf.function(experimental_relax_shapes=True)
    def skipgram_task(self, X, X_sum_neighbor, n_pid):

        my_pid, other_pid = tf.split(n_pid, [1, tf.shape(n_pid)[1]-1], axis=1)
        
        X_sum = tf.reduce_sum(X, axis=1)
        X_sum_other = tf.gather(X_sum, X.value_rowids()) - X.values

        
        X_sum_norm = tf.nn.l2_normalize(X_sum, axis=1)
        X_sum_other_norm = tf.nn.l2_normalize(X_sum_other, axis=1)
        X_sum_neighbor_norm = tf.slice(tf.gather(tf.nn.l2_normalize(X_sum_neighbor, axis=-1), X.value_rowids()), [0, 1, 0], [-1, -1, -1])
        X_value_norm = tf.nn.l2_normalize(X.values, axis=1)
        

        correct_cossim = tf.expand_dims(tf.einsum("Vd,Vd->V", X_value_norm, X_sum_other_norm), axis=1)
        wrong_cossim = tf.einsum("Vd,Nd->VN", X_value_norm, X_sum_norm)
        neighbor_cossim = tf.einsum("Vd,VNd->VN", X_value_norm, X_sum_neighbor_norm)
        cossim = tf.concat([correct_cossim, wrong_cossim, neighbor_cossim], axis=1)

        rowwise_mask = tf.expand_dims(tf.gather(X.row_lengths(), X.value_rowids()) > 1, axis=1)
        sameid_mask = tf.concat([tf.expand_dims(tf.cast(X.value_rowids(), "int32"), axis=1) != tf.expand_dims(tf.range(tf.shape(X.row_lengths())[0] + 1)-1, axis=0),
                                 tf.ones(tf.gather(tf.shape(X_sum_neighbor_norm), [0, 1], axis=0), dtype="bool")], axis=1)
        same_pid_mask = tf.concat([tf.gather(my_pid, X.value_rowids(), axis=0) != tf.linalg.matrix_transpose(my_pid),
                                   tf.gather(my_pid, X.value_rowids()) != tf.gather(other_pid, X.value_rowids())], axis=1)
        
        mask = tf.math.logical_and(tf.math.logical_and(rowwise_mask, sameid_mask), same_pid_mask)
        label = tf.concat([tf.ones(tf.shape(correct_cossim)), tf.zeros(tf.shape(wrong_cossim)), tf.zeros(tf.shape(neighbor_cossim))], axis=1)
        
        # Fixed AdaCos paramet2r
        scale = tf.sqrt(2.) * tf.math.log(tf.cast(tf.shape(label)[1], "float32") - 1.)

        label = tf.where(mask, label, tf.zeros(tf.shape(mask)))
        pred = tf.where(mask, scale * cossim, scale * -1.)
        loss = tf.math.reduce_mean(tf.keras.losses.categorical_crossentropy(label, pred, from_logits=True))
        return loss   
    
class ClassifyTrainModel(tf.keras.Model):
    
    def __init__(self, name_model, address_model, ix_emb_layer, cat_emb_layer):
        super(ClassifyTrainModel, self).__init__()
        self.name_model = name_model
        self.address_model = address_model
        self.ix_emb_layer = ix_emb_layer
        self.cat_emb_layer = cat_emb_layer
        self.dense = tf.keras.layers.Dense(cat_emb_layer.dim_categories, use_bias=True)
        self.scale = tf.sqrt(2.) * tf.math.log(tf.cast(cat_emb_layer.num_categories, "float32") - 1.)
        self.loss_function = SupervisedContrastiveLoss(from_logits=True)
        
        self.layer_norms = [tf.keras.layers.LayerNormalization() for i in range(3)]
        self.drop_out = tf.keras.layers.Dropout(0.1)
        self.nan_mask = tf.concat([tf.zeros([1, 1]), tf.ones([1, cat_emb_layer.num_categories-1])], axis=1)
        
    def transform(self, name, address, ix_values):
        X_name = self.layer_norms[0](tf.reduce_sum(self.drop_out(self.name_model(name)), axis=1))
        X_address = self.layer_norms[1](tf.reduce_sum(self.drop_out(self.address_model(address)), axis=1))
        X_ix = self.layer_norms[2](self.ix_emb_layer(ix_values))
        X = self.dense(tf.concat([X_name, X_address, X_ix], axis=-1))
        return X
        
    def call(self, name, address, ix_values, categories):
        X = self.transform(name, address, ix_values)
        classify_loss = self.classify_task(X, categories)
        return classify_loss
    
    def classify_task(self, X, C):
        cossim = tf.einsum("ND,CD->NC", tf.nn.l2_normalize(X, axis=1), tf.nn.l2_normalize(self.cat_emb_layer.category_dense, axis=1))
        
        true = tf.scatter_nd(tf.stack([C.value_rowids(), C.values], axis=1),
                              tf.ones(tf.shape(C.values)),
                              [tf.shape(X)[0], self.cat_emb_layer.num_categories])
        true = true * self.nan_mask
        calc_loc = tf.where(tf.reduce_sum(true, axis=1) > 0.)
        pred = self.scale * cossim
        loss = self.loss_function(tf.gather_nd(true, calc_loc), tf.gather_nd(pred, calc_loc))
        return loss

In [ ]:
class DataContainer():
    def __init__(self, df, ix_values, category_ix, positive_ix, neighbor_ix, neighbor_dist):
        self.name = tf.constant(df["sp_name"].values)
        self.address = tf.constant(df["sp_address"].values)
        self.position = tf.expand_dims(tf.constant(np.deg2rad(df[["latitude", "longitude"]].astype(np.float32).values), dtype="float32"), axis=0)
        self.pid = tf.constant(df["pid"].astype(np.int32).values, dtype="int32")
        self.group = tf.constant(df["group"].astype(np.int32).values, dtype="int32")
        self.ix_values = ix_values
        self.category_ix = category_ix
        self.positive_ix = positive_ix
        self.neighbor_ix = neighbor_ix
        self.neighbor_dist = neighbor_dist
        
    def get_position(self, ix):
        return tf.gather(tf.gather(self.position, ix, axis=1), 0)

    def call(self, ix, size=1):
        name = tf.gather(self.name, ix)
        address = tf.gather(self.address, ix)
        position = self.get_position(ix)
        categories = tf.gather(self.category_ix, ix)
        ix_values = {k: tf.gather(self.ix_values[k], ix) for k in self.ix_values.keys()}
        pids = tf.gather(self.pid, ix)
        return ix, name, address, position, ix_values, categories, pids, size
    
    @tf.function(experimental_relax_shapes=True)
    def log_haversine(self, X, Y):
        delta = Y - X
        x_lats = tf.gather(X, 0, axis=-1)
        y_lats = tf.gather(Y, 0, axis=-1)
        dlat = tf.gather(delta, 0, axis=-1)
        dlon = tf.gather(delta, 1, axis=-1)

        a = tf.sin(dlat/2) * tf.sin(dlat/2) + tf.cos(x_lats) * tf.cos(y_lats) * tf.sin(dlon/2) * tf.sin(dlon/2)
        c = tf.math.log(2 * tf.math.atan2(tf.sqrt(a), tf.sqrt(1-a)) + tf.keras.backend.epsilon())
        return c
    
    @tf.function(experimental_relax_shapes=True)
    def negative_sample_by_score_top_k(self, ix, score, except_loc=None, k=128, distcut=4):
        pos_ix = tf.gather(self.positive_ix, tf.gather(self.pid, ix))
        score = tf.tensor_scatter_nd_update(score,
                                            tf.stack([tf.cast(pos_ix.value_rowids(), "int32"), pos_ix.values], axis=1),
                                            tf.ones([tf.shape(pos_ix.values)[0]])*tf.float32.min/10.)
        if not except_loc is None:
            score = tf.tensor_scatter_nd_update(score,
                                                except_loc,
                                                tf.ones([tf.shape(except_loc)[0]])*tf.float32.min/10.)            
        score, neighbor = tf.math.top_k(score, k=k)
        _, order = tf.math.top_k(tf.random.uniform([tf.shape(ix)[0], k]), k=k)
        score = tf.gather(score, order, batch_dims=1)
        neighbor = tf.gather(neighbor, order, batch_dims=1)
        
        score = tf.reverse_sequence(score, 
                                    tf.ones([tf.shape(score)[0]], dtype="int32") * tf.shape(score)[1],
                                    seq_axis=1)
        neighbor = tf.reverse_sequence(neighbor,
                                  tf.ones([tf.shape(neighbor)[0]], dtype="int32") * tf.shape(neighbor)[1],
                                  seq_axis=1)
        return [score, neighbor]
    
    def get_positive_info(self, ix):
        pos_ix = tf.gather(self.positive_ix, tf.gather(self.pid, ix))
        Y = self.get_position(pos_ix)
        X = self.get_position(tf.gather(ix, pos_ix.value_rowids()))
        pos_dist = tf.RaggedTensor.from_value_rowids(self.log_haversine(X, Y.values), Y.value_rowids())
        pos_ix = pos_ix.to_tensor(default_value=-1)
        pos_ix = tf.where(tf.expand_dims(ix, axis=1) != pos_ix, pos_ix, -1)
        pos_dist = pos_dist.to_tensor(default_value=0.)
        return pos_dist, pos_ix
    
    @tf.function(experimental_relax_shapes=True)
    def random_walk_sampling(self, query_ix, step, top_k):
        query_ix = tf.cast(query_ix, "int32")
        searched = tf.expand_dims(query_ix, axis=1)
        current = tf.identity(query_ix)

        for i in range(step):
            n_ix = tf.gather(self.neighbor_ix, current)
            n_dist = tf.gather(self.neighbor_dist, current)
            n_dist = tf.where(tf.reduce_any(tf.expand_dims(n_ix, axis=2) == tf.expand_dims(searched, axis=1), axis=2), tf.keras.backend.epsilon(), n_dist)

            prob = n_dist / tf.reduce_sum(n_dist, axis=1, keepdims=True)
            logit = tf.math.log(prob) - tf.math.log(1-prob)        
            gumbel = logit - tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(logit))))
            nexts_ = tf.gather(n_ix, tf.math.top_k(gumbel, k=top_k)[1], batch_dims=1)
            searched = tf.concat([searched, nexts_], axis=1)
            current = tf.gather(nexts_, 0, axis=1)
        return tf.reshape(searched, [-1]), step*top_k + 1

In [ ]:
DIMSIZE = 256
WALK_STEP = 128
WALK_TOPK = 8
num_categories = max(category_ix_dict.values()) + 1

name_spe = SentencePieceEmbeddingLayer(32000, DIMSIZE, nadare_feature_dir + "sp_name.model", name_init_embedding)
address_spe = SentencePieceEmbeddingLayer(32000, DIMSIZE, nadare_feature_dir + "sp_address.model", address_init_embedding)

ix_emb_layer = IxEmbeddingLayer(ix_params_dict, num_layer=1, out_dim=DIMSIZE)
cat_emb_layer = CategoryEmbeddingLayer(num_categories, DIMSIZE)
#cat_emb_layer.category_dense.assign(category_init_embedding)

skipgram_model = SkipgramModel(name_spe)
#locrank_model = LocationModel(address_dpc, ix_emb_layer, num_rank=128)
classify_model = ClassifyTrainModel(name_spe, address_spe, ix_emb_layer, cat_emb_layer)
mixer_model = MixerModel(name_spe, address_spe, ix_emb_layer, cat_emb_layer, num_layer=3, num_dim=DIMSIZE)

mixer_loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [ ]:
container_cols = ["sp_name", "sp_address", "latitude", "longitude", "pid", "group"] 
train_container =  DataContainer(train_df[container_cols],
                                   train_ix_values,
                                    train_categories_ix,
                                    tf.cast(tf.ragged.constant(train_df.groupby("pid")["ix"].unique().tolist()), "int32"),
                                neighbor_ix,
                                neighbor_dist)

In [ ]:
train_ixs = tf.range(len(train_container.name), dtype="int64")
train_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .shuffle(len(train_ixs), reshuffle_each_iteration=True)\
                .batch(64, drop_remainder=True)\
                .map(lambda x: (train_container.call(x, 1), train_container.call(*train_container.random_walk_sampling(x, WALK_STEP, WALK_TOPK))), num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)\
                .prefetch(tf.data.AUTOTUNE)

In [ ]:
history = {"skipgram_loss": [], "classify_loss": [], "locrank_loss": [], "mixer_loss": []}
skipgram_loss = tf.keras.metrics.Mean(name='loss')
classify_loss = tf.keras.metrics.Mean(name='loss')
locrank_loss = tf.keras.metrics.Mean(name='loss')
mixer_loss = tf.keras.metrics.Mean(name='loss')

NUM_EPOCH = 1
skipgram_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
classify_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mixer_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)


@tf.function(experimental_relax_shapes=True)
def skipgram_forward_step(name, n_name, n_pid):
    with tf.GradientTape() as tape:
        loss = skipgram_model(name, n_name, n_pid, training=True)
    gradients = tape.gradient(loss, skipgram_model.trainable_variables)
    skipgram_optimizer.apply_gradients(zip(gradients, skipgram_model.trainable_variables))
    return loss

@tf.function(experimental_relax_shapes=True)
def classify_forward_step(name, address, ix_values, categories):
    with tf.GradientTape() as tape:
        loss = classify_model(name, address, ix_values, categories, training=True)
    gradients = tape.gradient(loss, classify_model.trainable_variables)
    if tf.reduce_all([tf.reduce_all(tf.math.is_finite(g)) for g in gradients]):
        classify_optimizer.apply_gradients(zip(gradients, classify_model.trainable_variables))
    return loss

#@tf.function(experimental_relax_shapes=True)
def locrank_forward_step(query_info, neighbor_info):
    with tf.GradientTape() as tape:
        loss = locrank_model(query_info, neighbor_info, training=True)
    gradients = tape.gradient(loss, locrank_model.trainable_variables)
    if tf.reduce_all([tf.reduce_all(tf.math.is_finite(g)) for g in gradients]):
        locrank_optimizer.apply_gradients(zip(gradients, locrank_model.trainable_variables))
    return loss

@tf.function(experimental_relax_shapes=True)
def mixer_forward_step(neighbor_info):
    with tf.GradientTape() as tape:
        loss = simcse_task(mixer_model, mixer_loss_func, neighbor_info)
    order = tape
    gradients = tape.gradient(loss, mixer_model.trainable_variables)
    if tf.reduce_all([tf.reduce_all(tf.math.is_finite(g)) for g in gradients]):
        mixer_optimizer.apply_gradients(zip(gradients, mixer_model.trainable_variables))
    return loss

with tqdm(total=NUM_EPOCH) as pbar:
    for epoch in range(NUM_EPOCH):
        step = 0
        for query_info, neighbor_info in train_ds:
            ix, name, address, position, ix_values, categories, pids, size = query_info
            n_name =  neighbor_info[1]
            n_pid = neighbor_info[6]
            n_size = neighbor_info[7]
            n_name = tf.reshape(n_name, [-1, n_size])
            n_pid =  tf.reshape(n_pid, [-1, n_size])
            sl = skipgram_forward_step(name, n_name, n_pid)
            cl = classify_forward_step(name, address, ix_values, categories)
            #ll = locrank_forward_step(query_info, neighbor_info)
            ml = mixer_forward_step(neighbor_info)
            skipgram_loss(sl)
            if not pd.isna(cl.numpy()):
                classify_loss(cl)
            #if not pd.isna(ll.numpy()):
            #    locrank_loss(ll)
            if not pd.isna(ml.numpy()):
                mixer_loss(ml)
            learning_text = "[{}/{}] ".format(str(step).zfill(5), len(train_ds))
            progress_text = "skipgram_loss: {:.4f} classify_loss: {:.4f} locrank_loss: {:.4f}, mixer_loss: {:.4f}".format(skipgram_loss.result().numpy(),
                                                                                                    classify_loss.result().numpy(),
                                                                                                    locrank_loss.result().numpy(),
                                                                                                    mixer_loss.result().numpy(),
                                                                                                    )
            pbar.set_postfix_str(learning_text + progress_text)
            step += 1
        history["skipgram_loss"].append(skipgram_loss.result().numpy())
        history["classify_loss"].append(classify_loss.result().numpy())
        history["locrank_loss"].append(locrank_loss.result().numpy())
        history["mixer_loss"].append(mixer_loss.result().numpy())
        print(progress_text)

        skipgram_loss.reset_states()
        classify_loss.reset_states()
        locrank_loss.reset_states()
        mixer_loss.reset_states()
        pbar.update(1)
        classify_model.save_weights(f"../feature/FEAT24/classify_model_{epoch}")
        mixer_model.save_weights(f"../feature/FEAT24/mixer_model_{epoch}")

  0%|          | 0/1 [00:00<?, ?it/s]

skipgram_loss: 6.2974 classify_loss: 2.6489 locrank_loss: 0.0000, mixer_loss: 5.4179


In [ ]:
pd.DataFrame(history)

,skipgram_loss,classify_loss,locrank_loss,mixer_loss
0,3.947135,3.118746,0.0,5.808200
1,3.582191,2.265229,0.0,5.777259
2,3.459677,2.113289,0.0,5.770914


In [ ]:
pd.DataFrame(history)

,skipgram_loss,classify_loss,locrank_loss,mixer_loss
0,6.297405,2.648874,0.0,5.417853


In [ ]:
train_df["true_count"] = train_df.groupby("pid")["ix"].transform("count")

In [ ]:
# name
train_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(train_df)))\
                .batch(1024)\
                .map(train_container.call)

embeddings = []
for ix, name, address, position, ix_values, categories, _, _ in tqdm(train_ds):
    embeddings.append(tf.nn.l2_normalize(tf.reduce_sum(name_spe(name), axis=1), axis=1))
name_embeddings = tf.nn.l2_normalize(tf.concat(embeddings, axis=0), axis=1)

# embedding only
emb_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .batch(128)

neighbors = []
for ixs in tqdm(emb_ds):
    emb = tf.gather(name_embeddings, ixs)
    score, indices = tf.math.top_k(tf.einsum("md,nd->mn", emb, name_embeddings), k=16)
    neighbors.append(indices)
neighbors = tf.minimum(tf.concat(neighbors, axis=0).numpy()[:len(train_df)], len(train_df)-1)

print("precision@16", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors]).sum(axis=1) / train_df["true_count"]).mean())

  0%|          | 0/1113 [00:00<?, ?it/s]

  0%|          | 0/8897 [00:00<?, ?it/s]

precision@16 0.7794088383447885


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(train_df)))\
                .batch(1024)\
                .map(train_container.call)



embeddings = []
for query_info in tqdm(train_ds):
    embeddings.append(mixer_model(query_info))
mix_embeddings = tf.nn.l2_normalize(tf.concat(embeddings, axis=0), axis=1)
                      
# embedding only
emb_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .batch(128)

neighbors = []
for ixs in tqdm(emb_ds):
    emb = tf.gather(mix_embeddings, ixs)
    score, indices = tf.math.top_k(tf.einsum("md,nd->mn", emb, mix_embeddings), k=16)
    neighbors.append(indices)
neighbors = tf.minimum(tf.concat(neighbors, axis=0).numpy()[:len(train_df)], len(train_df)-1)

print("precision@16", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors]).sum(axis=1) / train_df["true_count"]).mean())

  0%|          | 0/1113 [00:00<?, ?it/s]

  0%|          | 0/8897 [00:00<?, ?it/s]

precision@16 0.9011579694686475


In [ ]:
normalized_category_dense = tf.nn.l2_normalize(cat_emb_layer.category_dense, axis=1)
pseudo_categories = []
category_ix_dict_r = {v: k for k, v in category_ix_dict.items()}
eval_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .batch(1024)\
                .map(train_container.call)

for ix, name, address, position, ix_values, categories, pid, size in tqdm(eval_ds):
    X = tf.nn.l2_normalize(classify_model.transform(name, address, ix_values), axis=1)
    label = tf.argmax(tf.einsum("nd,md->nm", X, normalized_category_dense), axis=1)
    for l in label.numpy():
        pseudo_categories.append("nan, " + category_ix_dict_r[l])
        
def remove_test_only_category(text):
    res = []
    for x in text.split(", "):
        if x in category_ix_dict.keys():
            res.append(x)
    return ", ".join(res)

train_df["categories"] = np.vectorize(remove_test_only_category)(train_df["categories"])
train_df["categories"] = np.where(train_df["categories"] == "nan", pseudo_categories, train_df["categories"])
train_categories_ix = get_category_ix(train_df, category_ix_dict)

  0%|          | 0/1113 [00:00<?, ?it/s]

In [ ]:
train_container =  DataContainer(train_df[container_cols],
                                   train_ix_values,
                                    train_categories_ix,
                                    tf.cast(tf.ragged.constant(train_df.groupby("pid")["ix"].unique().tolist()), "int32"),
                                neighbor_ix,
                                neighbor_dist)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(train_df)))\
                .batch(1024)\
                .map(train_container.call)



embeddings = []
for query_info in tqdm(train_ds):
    embeddings.append(mixer_model(query_info))
mix_embeddings = tf.nn.l2_normalize(tf.concat(embeddings, axis=0), axis=1)
                      
# embedding only
emb_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .batch(128)

neighbors = []
for ixs in tqdm(emb_ds):
    emb = tf.gather(mix_embeddings, ixs)
    score, indices = tf.math.top_k(tf.einsum("md,nd->mn", emb, mix_embeddings), k=16)
    neighbors.append(indices)
neighbors = tf.minimum(tf.concat(neighbors, axis=0).numpy()[:len(train_df)], len(train_df)-1)

print("precision@16", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors]).sum(axis=1) / train_df["true_count"]).mean())

  0%|          | 0/1113 [00:00<?, ?it/s]

  0%|          | 0/8897 [00:00<?, ?it/s]

precision@16 0.9042013831103947


In [ ]:
category_embeddings = tf.nn.l2_normalize(tf.math.reduce_sum(tf.gather(cat_emb_layer.category_dense, train_categories_ix), axis=1), axis=1)

# embedding only
emb_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .batch(128)

neighbors = []
for ixs in tqdm(emb_ds):
    emb = tf.gather(category_embeddings, ixs)
    score, indices = tf.math.top_k(tf.einsum("md,nd->mn", emb, category_embeddings), k=16)
    neighbors.append(indices)
neighbors = tf.minimum(tf.concat(neighbors, axis=0).numpy()[:len(train_df)], len(train_df)-1)

print("precision@16", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors]).sum(axis=1) / train_df["true_count"]).mean())


  0%|          | 0/8897 [00:00<?, ?it/s]

precision@16 0.07047811507569895


In [ ]:
# address
train_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(train_df)))\
                .batch(1024)\
                .map(train_container.call)

embeddings = []
for ix, address, address, position, ix_values, categories, _, _ in tqdm(train_ds):
    embeddings.append(tf.nn.l2_normalize(tf.reduce_sum(address_spe(address), axis=1), axis=1))
address_embeddings = tf.nn.l2_normalize(tf.concat(embeddings, axis=0), axis=1)

# embedding only
emb_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .batch(128)

neighbors = []
for ixs in tqdm(emb_ds):
    emb = tf.gather(address_embeddings, ixs)
    score, indices = tf.math.top_k(tf.einsum("md,nd->mn", emb, address_embeddings), k=16)
    neighbors.append(indices)
neighbors = tf.minimum(tf.concat(neighbors, axis=0).numpy()[:len(train_df)], len(train_df)-1)

print("precision@16", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors]).sum(axis=1) / train_df["true_count"]).mean())

  0%|          | 0/1113 [00:00<?, ?it/s]

  0%|          | 0/8897 [00:00<?, ?it/s]

precision@16 0.8718289779802572


In [ ]:
# ix_values
train_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(train_df)))\
                .batch(1024)\
                .map(train_container.call)

embeddings = []
for ix, ix_values, ix_values, position, ix_values, categories, _, _ in tqdm(train_ds):
    embeddings.append(tf.nn.l2_normalize(ix_emb_layer(ix_values), axis=1))
ix_values_embeddings = tf.nn.l2_normalize(tf.concat(embeddings, axis=0), axis=1)

# embedding only
emb_ds = tf.data.Dataset.from_tensor_slices(train_ixs)\
                .batch(128)

neighbors = []
for ixs in tqdm(emb_ds):
    emb = tf.gather(ix_values_embeddings, ixs)
    score, indices = tf.math.top_k(tf.einsum("md,nd->mn", emb, ix_values_embeddings), k=16)
    neighbors.append(indices)
neighbors = tf.minimum(tf.concat(neighbors, axis=0).numpy()[:len(train_df)], len(train_df)-1)

print("precision@16", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors]).sum(axis=1) / train_df["true_count"]).mean())

  0%|          | 0/1113 [00:00<?, ?it/s]

  0%|          | 0/8897 [00:00<?, ?it/s]

precision@16 0.1810690153365134


In [ ]:
# simple version

import tensorflow_ranking as tfr

def haversine(X, Y):
    delta = Y - X
    x_lats = tf.gather(X, 0, axis=-1)
    y_lats = tf.gather(Y, 0, axis=-1)
    dlat = tf.gather(delta, 0, axis=-1)
    dlon = tf.gather(delta, 1, axis=-1)

    a = tf.sin(dlat/2) * tf.sin(dlat/2) + tf.cos(x_lats) * tf.cos(y_lats) * tf.sin(dlon/2) * tf.sin(dlon/2)
    c = 2 * tf.math.atan2(tf.sqrt(a), tf.sqrt(1-a))
    return c

def approx_ranks(logits):
    r"""Computes approximate ranks given a list of logits.
    Given a list of logits, the rank of an item in the list is one plus the total
    number of items with a larger logit. In other words,
    rank_i = 1 + \sum_{j \neq i} I_{s_j > s_i},
    where "I" is the indicator function. The indicator function can be
    approximated by a generalized sigmoid:
    I_{s_j < s_i} \approx 1/(1 + exp(-(s_j - s_i)/temperature)).
    This function approximates the rank of an item using this sigmoid
    approximation to the indicator function. This technique is at the core
    of "A general approximation framework for direct optimization of
    information retrieval measures" by Qin et al.
    Args:
    logits: A `Tensor` with shape [batch_size, list_size]. Each value is the
      ranking score of the corresponding item.
    Returns:
    A `Tensor` of ranks with the same shape as logits.
    """
    list_size = tf.shape(input=logits)[1]
    x = tf.tile(tf.expand_dims(logits, 2), [1, 1, list_size])
    y = tf.tile(tf.expand_dims(logits, 1), [1, list_size, 1])
    pairs = tf.sigmoid(y - x)
    return tf.reduce_sum(input_tensor=pairs, axis=-1) + .5
    
    
class ApproxMSE(tf.keras.layers.Layer):
    def __init__(self,from_logits=True, alpha=1.):
        super(ApproxMSE, self).__init__()
        self.alpha = alpha
        if not from_logits:
            NotImplementedError
    
    def call(self, y_true, y_pred, sample_weight=None):
        if sample_weight is None:
            sample_weight = tf.ones(tf.shape(y_pred), dtype=tf.float32)
        
        K = tf.cast(tf.shape(y_true)[1], "float32")    
        rank = approx_ranks(y_pred * self.alpha)
        label_sum = tf.reduce_sum(y_true, axis=1, keepdims=True)
        rmse = tf.reduce_sum(tf.maximum(0., rank - label_sum) * y_true, axis=1, keepdims=True) / (label_sum + tf.keras.backend.epsilon())
        
        return tf.reduce_sum(rmse * sample_weight) / tf.reduce_sum(sample_weight)

class LogisticModel(tf.keras.Model):
    def __init__(self, dim):
        super(LogisticModel, self).__init__()
        self.scale = tf.Variable(tf.zeros([dim], dtype="float32"), trainable=True)
        self.loss_func = ApproxMSE()#tfr.keras.losses.ApproxNDCGLoss()
        
    def call(self, X, Y):
        logit = tf.einsum("nmd,d->nm", X, self.scale)# + self.bias
        loss = self.loss_func(Y, logit)
        return loss    
    
    
from sklearn.feature_extraction.text import TfidfVectorizer
bagofword_vectorizer = TfidfVectorizer(min_df=0, binary=True, use_idf=False, norm="l2", dtype=np.float32)
name_bow = bagofword_vectorizer.fit_transform(train_df["sp_name"]).tocoo()

name_bow_rag_data = tf.RaggedTensor.from_value_rowids(name_bow.data, name_bow.row)
name_bow_rag_col = tf.RaggedTensor.from_value_rowids(name_bow.col, name_bow.row)

name_bow_coo = tf.SparseTensor(tf.cast(tf.stack([name_bow.row, name_bow.col], axis=-1), "int64"), name_bow.data, name_bow.shape)

In [ ]:
simple_logistic_model = LogisticModel(5)
history = {"logistic_loss": []}
logistic_loss = tf.keras.metrics.Mean(name='loss')
logistic_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

train_ixs = tf.range(len(train_container.name))
train_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(train_df)))\
                .shuffle(len(train_ixs), reshuffle_each_iteration=True)\
                .batch(32, drop_remainder=True)\
                .map(train_container.call)

@tf.function(experimental_relax_shapes=True)
def logistic_forward_step(X, Y):
    with tf.GradientTape() as tape:
        loss = simple_logistic_model(X, Y, training=True)
    gradients = tape.gradient(loss, simple_logistic_model.trainable_variables)
    if tf.reduce_all([tf.reduce_all(tf.math.is_finite(g)) for g in gradients]):
        logistic_optimizer.apply_gradients(zip(gradients, simple_logistic_model.trainable_variables))
    return loss

@tf.function(experimental_relax_shapes=True)
def get_candidates(query_info, move_scale):
    ix, name, address, position, ix_values, categories, pid, size = query_info
    X = tf.expand_dims(train_container.get_position(ix), axis=1)
    dist = train_container.log_haversine(X, train_container.position)
    name_cossim = tf.einsum("nd,md->nm", tf.gather(name_embeddings, ix), name_embeddings)
    address_cossim = tf.einsum("nd,md->nm", tf.gather(address_embeddings, ix), address_embeddings)
    category_cossim = tf.einsum("nd,md->nm", tf.gather(category_embeddings, ix), category_embeddings)

    #coo = (name_bow[ix] * name_bow.T).tocoo()
    #sparse_name_sim = tf.scatter_nd(tf.stack([coo.row, coo.col], axis=-1), coo.data, [len(ix), len(train_df)])
    indices = tf.stack([tf.repeat(tf.range(len(ix)), tf.gather(name_bow_rag_data, ix).row_lengths()),
                        tf.gather(name_bow_rag_col, ix).values], axis=-1)
    values = tf.gather(name_bow_rag_data, ix).values
    shape = [len(ix), name_bow_coo.shape[1]]
    query = tf.scatter_nd(indices, values, shape)
    sparse_name_sim = tf.sparse.sparse_dense_matmul(query, name_bow_coo, adjoint_b=True)

    score = tf.einsum("d,nmd->nm", move_scale, tf.stack([dist, sparse_name_sim, name_cossim, address_cossim, category_cossim], axis=-1))
    #except_loc = tf.reshape(tf.stack([tf.repeat(tf.expand_dims(tf.range(len(X)), axis=1), 4, axis=1),
    #                tf.math.top_k(-dist, k=4)[1]], axis=-1), [-1, 2])


    neg_dist, neg_ixs = train_container.negative_sample_by_score_top_k(ix, score, except_loc=None, k=1024)

    pos_dist, pos_ixs = train_container.get_positive_info(ix)
    pos_order = tf.slice(tf.argsort(pos_dist, axis=1), [0, 0], [-1, tf.minimum(tf.shape(pos_dist)[1], 64)])
    pos_dist = tf.gather(pos_dist, pos_order, batch_dims=1)
    pos_ixs = tf.gather(pos_ixs, pos_order, batch_dims=1)
    pos_ix = tf.where(tf.expand_dims(ix, axis=1) != pos_ixs, pos_ixs, -1)

    update_ixs = tf.where(pos_ixs >= 0)
    target_ixs = tf.tensor_scatter_nd_update(neg_ixs, update_ixs, tf.gather_nd(pos_ixs, update_ixs))
    target_label = tf.tensor_scatter_nd_update(tf.zeros(tf.shape(neg_dist)), update_ixs, tf.ones([tf.shape(update_ixs)[0]]))
    target_dist = tf.gather(dist, target_ixs, batch_dims=1)
    target_name_cossim = tf.gather(name_cossim, target_ixs, batch_dims=1)
    target_sparse_name_sim = tf.gather(sparse_name_sim, target_ixs, batch_dims=1)
    target_address_cossim = tf.gather(address_cossim, target_ixs, batch_dims=1)
    target_category_cossim = tf.gather(category_cossim, target_ixs, batch_dims=1)

    target_feat = tf.stack([target_dist, target_sparse_name_sim, target_name_cossim, target_address_cossim, target_category_cossim], axis=-1)
    return target_feat, target_label     

move_scale = tf.convert_to_tensor([-1., 0., 0., 0., 0.], dtype="float32")

with tqdm(total=len(train_ds)) as pbar:
    step = 0
    for query_info in train_ds:
        target_feat, target_label = get_candidates(query_info, move_scale)
        loss = logistic_forward_step(target_feat, target_label)
        move_scale = move_scale * .99 + (simple_logistic_model.scale * (1. - .99))
        if tf.math.is_finite(loss):
            logistic_loss(loss)
        progress_text = "logistic_loss: {:.4f}".format(logistic_loss.result().numpy())
        pbar.set_postfix_str(progress_text)
        pbar.update(1)
        step += 1

  0%|          | 0/35587 [00:00<?, ?it/s]

In [ ]:
print(progress_text)
simple_logistic_model.scale

logistic_loss: 8.1173


<tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=
array([-4.289177 , 10.927236 , 24.742352 ,  8.860054 ,  6.9624615],
      dtype=float32)>

In [ ]:
# embedding + haversine
train_ixs_ds = tf.data.Dataset.from_tensor_slices(tf.range(len(train_df)))\
                .batch(16)

neighbors = []
dist_ranks = []

@tf.function(experimental_relax_shapes=True)
def get_indices(ix):
    X = tf.expand_dims(train_container.get_position(ix), axis=1)
    dist = train_container.log_haversine(X, train_container.position)
    indices = tf.stack([tf.repeat(tf.range(len(ix)), tf.gather(name_bow_rag_data, ix).row_lengths()),
                        tf.gather(name_bow_rag_col, ix).values], axis=-1)
    values = tf.gather(name_bow_rag_data, ix).values
    shape = [len(ix), name_bow_coo.shape[1]]
    query = tf.scatter_nd(indices, values, shape)
    sparse_name_sim = tf.sparse.sparse_dense_matmul(query, name_bow_coo, adjoint_b=True)
    
    name_cossim = tf.einsum("nd,md->nm", tf.gather(name_embeddings, ix), name_embeddings)
    category_cossim = tf.einsum("nd,md->nm", tf.gather(category_embeddings, ix), category_embeddings)
    address_cossim = tf.einsum("nd,md->nm", tf.gather(address_embeddings, ix), address_embeddings)
    score = tf.einsum("d,nmd->nm", simple_logistic_model.scale, tf.stack([dist, sparse_name_sim, name_cossim, address_cossim, category_cossim], axis=-1))
    
    _, indices = tf.math.top_k(score, k=100)
    return indices
    
for ix in tqdm(train_ixs_ds):
    indices = get_indices(ix)
    #dist_rank = tf.gather(tf.argsort(tf.argsort(-dist, axis=1), axis=1), indices, batch_dims=-1)
    #dist_ranks.append(dist_rank)
    neighbors.append(indices)

neighbors = tf.minimum(tf.concat(neighbors, axis=0).numpy()[:len(train_df)], len(train_df)-1).numpy()
print("precision@16", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors[:, :16]]).sum(axis=1) / train_df["true_count"]).mean())
print("precision@100", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors[:, :100]]).sum(axis=1) / train_df["true_count"]).mean())


  0%|          | 0/71176 [00:00<?, ?it/s]

precision@16 0.9724738386633075
precision@100 0.9900192900391731


In [ ]:
import gc
gc.collect()

print("precision@20", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors[:, :20]]).sum(axis=1) / train_df["true_count"]).mean())
print("precision@30", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors[:, :30]]).sum(axis=1) / train_df["true_count"]).mean())
print("precision@50", ((train_df["pid"].values.reshape(-1, 1) == train_df["pid"].values[neighbors[:, :50]]).sum(axis=1) / train_df["true_count"]).mean())


precision@20 0.9756721880668088
precision@30 0.9805842104666077
precision@50 0.9852901910022868


In [ ]:



class TFMiniBatchHKmeans(tf.keras.Model):
    def __init__(self, n_cluster=64, learning_rate=0.1):
        super(TFMiniBatchHKmeans, self).__init__()
        self.n_cluster = n_cluster
        self.learning_rate = learning_rate
        self.centroids = tf.Variable(tf.zeros((n_cluster, 2), dtype="float32"), trainable=False)
    
    def init_centroid(self, X):
        new_centroids = []
        choosed = tf.zeros(len(X), dtype="bool")

        ix = tf.cast(tf.random.uniform([1], maxval=len(X))//1, "int32")[0]
        choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
        new_centroids.append(tf.gather(X, ix))
        dist = tf.ones([tf.shape(X)[0]]) * tf.float32.max / 10

        for i in tqdm(range(self.n_cluster-1)):
            centroid = tf.gather(X, ix)
    
            dist = tf.where(choosed, tf.keras.backend.epsilon() , tf.minimum(dist, tf.math.square(hkmeans.haversine(centroid, X))))
            prob = dist / tf.reduce_sum(dist, axis=0, keepdims=True)
            logit = tf.math.log(prob) - tf.math.log(1-prob)        
            gumbel = logit - tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(logit))))
            ix = tf.cast(tf.argmax(gumbel), "int32")
            choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
            new_centroids.append(tf.gather(X, ix))
        self.centroids.assign(tf.stack(new_centroids, axis=0))
        
    def partial_fit(self, X):
        score = self.haversine(tf.expand_dims(X, axis=1), tf.expand_dims(self.centroids, axis=0))
        maxix = tf.argmin(score, axis=1)
        new_centroids = tf.math.unsorted_segment_mean(X, maxix, num_segments=self.n_cluster)
        num_member = tf.math.unsorted_segment_sum(tf.ones((len(X), 1)), maxix, num_segments=self.n_cluster)
        new_centroids = tf.where(num_member > 0, new_centroids, self.centroids)
        new_centroids = new_centroids * self.learning_rate + self.centroids * (1 - self.learning_rate)
        dist = tf.reduce_mean(tf.reduce_min(score, axis=1))
        self.centroids.assign(new_centroids)
        return dist
    
    def haversine(self, X, Y):
        delta = Y - X
        x_lats = tf.gather(X, 0, axis=-1)
        y_lats = tf.gather(Y, 0, axis=-1)
        dlat = tf.gather(delta, 0, axis=-1)
        dlon = tf.gather(delta, 1, axis=-1)

        a = tf.sin(dlat/2) * tf.sin(dlat/2) + tf.cos(x_lats) * tf.cos(y_lats) * tf.sin(dlon/2) * tf.sin(dlon/2)
        c = tf.math.atan2(tf.sqrt(a), tf.sqrt(1-a))
        return c
    
    def transform(self, X):
        res = tf.argmin(self.haversine(tf.expand_dims(X, axis=1), tf.expand_dims(self.centroids, axis=0)), axis=-1)
        return res

class TFMiniBatchSKmeans(tf.keras.Model):
    def __init__(self, n_cluster=64, n_dim=100, alpha=3/2, learning_rate=0.1):
        super(TFMiniBatchSKmeans, self).__init__()
        self.n_dim = n_dim
        self.n_cluster = n_cluster
        self.alpha = alpha
        self.learning_rate = learning_rate

        self.centroids = tf.Variable(tf.zeros((n_cluster, n_dim), dtype="float32"), trainable=False)
    
    def init_centroid(self, X):
        new_centroids = []
        X = tf.nn.l2_normalize(X, axis=-1)
        choosed = tf.zeros(len(X), dtype="bool")

        ix = tf.cast(tf.random.uniform([1], maxval=len(X))//1, "int32")[0]
        choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
        new_centroids.append(tf.gather(X, ix))
        
        dist = tf.ones([tf.shape(X)[0]]) * tf.float32.max / 10

        for i in tqdm(range(self.n_cluster-1)):
            centroid = tf.gather(X, ix)
            cos = tf.einsum("d,nd->n", centroid, X)
            dist = tf.where(choosed, tf.keras.backend.epsilon() , tf.maximum(tf.keras.backend.epsilon(), tf.minimum(dist, 1. - cos)))         
            prob = dist / tf.reduce_sum(dist, axis=0, keepdims=True)
            logit = tf.math.log(prob) - tf.math.log(1-prob)        
            gumbel = logit - tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(logit))))
            ix = tf.cast(tf.argmax(gumbel), "int32")
            choosed = tf.where(tf.range(len(X)) == ix, True, choosed)
            new_centroids.append(X[ix])
        self.centroids.assign(tf.stack(new_centroids, axis=0))
        
    @tf.function
    def partial_fit(self, X):
        score = tf.einsum("nd,kd->nk", X, self.centroids)
        maxix = tf.argmax(score, axis=1)
        new_centroids = tf.math.unsorted_segment_mean(X, maxix, num_segments=self.n_cluster)
        num_member = tf.math.unsorted_segment_sum(tf.ones((len(X), 1)), maxix, num_segments=self.n_cluster)
        new_centroids = tf.where(num_member > 0, new_centroids, self.centroids)
        new_centroids = tf.nn.l2_normalize(new_centroids, axis=1)
        new_centroids = new_centroids * self.learning_rate + self.centroids * (1 - self.learning_rate)
        new_centroids = tf.nn.l2_normalize(new_centroids, axis=1)
        dist = tf.reduce_mean(self.alpha - tf.reduce_max(score, axis=1))
        self.centroids.assign(new_centroids)
        return dist
    
    def transform(self, X):
        res = tf.argmax(tf.einsum("...d,kd->...k", X, self.centroids), axis=-1)
        return res

In [ ]:
hkmeans = TFMiniBatchHKmeans(255*8, learning_rate=1.)
hkmeans.init_centroid(train_container.position[0])

scores = []
with tqdm(total=2000) as pbar:
    for _ in range(2000):
        ixs = np.random.choice(train_ixs, 100000, False)
        score = hkmeans.partial_fit(tf.gather(train_container.position[0], ixs))
        progress_text = "{:4f}".format(score.numpy())
        pbar.set_postfix_str(progress_text)    
        pbar.update(1)
        
centroid_position = hkmeans.centroids.numpy()
data = {}
data["distance_matrix"] = (haversine(tf.expand_dims(centroid_position, axis=1), tf.expand_dims(centroid_position, axis=0)).numpy() * 1e9).astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
hkmeans.centroids.assign(tf.gather(hkmeans.centroids, routes))

  0%|          | 0/2039 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

<tf.Variable 'UnreadVariable' shape=(2040, 2) dtype=float32, numpy=
array([[ 9.4021484e-02, -9.3767560e-01],
       [ 3.9109762e-04, -8.9150327e-01],
       [-5.5816524e-02, -9.1121477e-01],
       ...,
       [ 1.5525542e-01, -1.1209325e+00],
       [ 1.4277864e-01, -1.0969130e+00],
       [ 4.9140606e-02, -1.0596082e+00]], dtype=float32)>

In [ ]:
category_embedding = tf.nn.l2_normalize(cat_emb_layer.category_dense, axis=1)

data = {}
data["distance_matrix"] = ((1 - tf.einsum("nd,md->nm", category_embedding, category_embedding)) / 2. * (1. - tf.eye(tf.shape(category_embedding)[0])) * 1e9).numpy().astype(np.int64).tolist()
data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
# center frequent value
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]

category_df = pd.DataFrame({"category": [category_ix_dict_r[id_] for id_ in routes]})
category_df["category_ix"] = category_df["category"].map(category_ix_dict)
category_df["category_tsp_ix"] = category_df.index

In [ ]:
name_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(name_embeddings)[1], learning_rate=1.)
name_kmeans.init_centroid(tf.gather(name_embeddings, train_df.drop_duplicates("name").index))
with tqdm(total=1000) as pbar:
    for _ in range(1000):
        ixs = np.random.choice(train_ixs, 100000, False)
        score = name_kmeans.partial_fit(tf.gather(name_embeddings, ixs))
        progress_text = "{:4f}".format(score.numpy())
        pbar.set_postfix_str(progress_text)    
        pbar.update(1)

data = {}
data["distance_matrix"] = ((1. - tf.einsum("nd,md->nm", name_kmeans.centroids, name_kmeans.centroids)) * 1e9).numpy().astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
name_kmeans.centroids.assign(tf.gather(name_kmeans.centroids, routes))

  0%|          | 0/2039 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

<tf.Variable 'UnreadVariable' shape=(2040, 256) dtype=float32, numpy=
array([[ 0.05367883, -0.09877654, -0.0362552 , ..., -0.02462533,
         0.02518191,  0.04509797],
       [-0.03846163, -0.07003859, -0.0143156 , ..., -0.07688364,
         0.06933416,  0.0271528 ],
       [ 0.05181199,  0.0245278 ,  0.00708091, ..., -0.02643215,
         0.11937096, -0.02803057],
       ...,
       [ 0.03661745, -0.02569252,  0.02217283, ..., -0.05264423,
        -0.10504616,  0.02601274],
       [ 0.07170249, -0.08004361, -0.01019648, ..., -0.06460643,
        -0.02874802,  0.01596966],
       [ 0.0722662 , -0.0015444 , -0.02889389, ..., -0.04342653,
         0.01560971,  0.01888697]], dtype=float32)>

In [ ]:
address_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(address_embeddings)[1], learning_rate=1.)
address_kmeans.init_centroid(tf.gather(address_embeddings, train_df.drop_duplicates("address").index))
with tqdm(total=1000) as pbar:
    for _ in range(1000):
        ixs = np.random.choice(train_ixs, 100000, False)
        score = address_kmeans.partial_fit(tf.gather(address_embeddings, ixs))
        progress_text = "{:4f}".format(score.numpy())
        pbar.set_postfix_str(progress_text)    
        pbar.update(1)

data = {}
data["distance_matrix"] = ((1. - tf.einsum("nd,md->nm", address_kmeans.centroids, address_kmeans.centroids)) * 1e9).numpy().astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
address_kmeans.centroids.assign(tf.gather(address_kmeans.centroids, routes))

  0%|          | 0/2039 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

<tf.Variable 'UnreadVariable' shape=(2040, 256) dtype=float32, numpy=
array([[-0.01540297, -0.10685989,  0.04598546, ..., -0.03650163,
         0.0284561 ,  0.1487307 ],
       [-0.07576877, -0.0263731 ,  0.03517479, ...,  0.08155105,
         0.00445443, -0.03914481],
       [-0.04656794,  0.06876873, -0.06128385, ..., -0.0618774 ,
         0.05225549,  0.00664888],
       ...,
       [-0.11305363,  0.04342153, -0.02193956, ...,  0.06300627,
         0.00578293, -0.04309898],
       [-0.01640577,  0.12669231, -0.01856684, ...,  0.03727403,
         0.0161635 , -0.08416845],
       [-0.01588172,  0.02062212, -0.01316744, ...,  0.07660025,
         0.0980431 , -0.04491289]], dtype=float32)>

In [ ]:
category_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(category_embeddings)[1], learning_rate=1.)
category_kmeans.init_centroid(tf.gather(category_embeddings, train_df.drop_duplicates("categories").index))
with tqdm(total=1000) as pbar:
    for _ in range(1000):
        ixs = np.random.choice(train_ixs, 100000, False)
        score = category_kmeans.partial_fit(tf.gather(category_embeddings, ixs))
        progress_text = "{:4f}".format(score.numpy())
        pbar.set_postfix_str(progress_text)    
        pbar.update(1)

data = {}
data["distance_matrix"] = ((1. - tf.einsum("nd,md->nm", category_kmeans.centroids, category_kmeans.centroids)) * 1e9).numpy().astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
category_kmeans.centroids.assign(tf.gather(category_kmeans.centroids, routes))

  0%|          | 0/2039 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

<tf.Variable 'UnreadVariable' shape=(2040, 256) dtype=float32, numpy=
array([[-0.03490861,  0.07001233,  0.07232594, ...,  0.08803911,
        -0.03294925, -0.0022645 ],
       [-0.01074211,  0.07262552, -0.01140475, ...,  0.02053834,
        -0.11359691, -0.01559312],
       [ 0.04806343,  0.09930054, -0.00049694, ...,  0.04805629,
        -0.07743084, -0.03839194],
       ...,
       [-0.1065522 ,  0.00829812,  0.05307719, ..., -0.02529685,
        -0.01690373, -0.00807665],
       [-0.04488681, -0.00280879,  0.03796248, ..., -0.02585155,
        -0.06523383, -0.03156277],
       [-0.07051554,  0.00851297,  0.0908815 , ...,  0.05799394,
        -0.00529168,  0.00387569]], dtype=float32)>

In [ ]:
ix_values_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(ix_values_embeddings)[1], learning_rate=1.)
ix_values_kmeans.init_centroid(tf.gather(ix_values_embeddings, train_df.index))
with tqdm(total=1000) as pbar:
    for _ in range(1000):
        ixs = np.random.choice(train_ixs, 100000, False)
        score = ix_values_kmeans.partial_fit(tf.gather(ix_values_embeddings, ixs))
        progress_text = "{:4f}".format(score.numpy())
        pbar.set_postfix_str(progress_text)    
        pbar.update(1)

data = {}
data["distance_matrix"] = ((1. - tf.einsum("nd,md->nm", ix_values_kmeans.centroids, ix_values_kmeans.centroids)) * 1e9).numpy().astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
ix_values_kmeans.centroids.assign(tf.gather(ix_values_kmeans.centroids, routes))

  0%|          | 0/2039 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

<tf.Variable 'UnreadVariable' shape=(2040, 256) dtype=float32, numpy=
array([[-0.02488252, -0.00290189, -0.00338166, ...,  0.01909287,
        -0.03749029,  0.00595636],
       [-0.02683397, -0.00560614, -0.00079461, ...,  0.02416875,
        -0.04125655,  0.0053868 ],
       [-0.02655462, -0.00474923, -0.00684417, ...,  0.02659443,
        -0.05218446, -0.00306095],
       ...,
       [ 0.0024772 ,  0.00661306, -0.0056834 , ...,  0.04607531,
        -0.02661628,  0.02193001],
       [ 0.00315666,  0.00671522, -0.00205756, ...,  0.04134945,
        -0.02631197,  0.01930606],
       [-0.00847554, -0.00494793,  0.0020312 , ...,  0.03560913,
        -0.02989377,  0.01253536]], dtype=float32)>

In [ ]:
mix_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(mix_embeddings)[1], learning_rate=1.)
mix_kmeans.init_centroid(tf.gather(mix_embeddings, train_df.index))
with tqdm(total=1000) as pbar:
    for _ in range(1000):
        ixs = np.random.choice(train_ixs, 100000, False)
        score = mix_kmeans.partial_fit(tf.gather(mix_embeddings, ixs))
        progress_text = "{:4f}".format(score.numpy())
        pbar.set_postfix_str(progress_text)    
        pbar.update(1)

data = {}
data["distance_matrix"] = ((1. - tf.einsum("nd,md->nm", mix_kmeans.centroids, mix_kmeans.centroids)) * 1e9).numpy().astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
mix_kmeans.centroids.assign(tf.gather(mix_kmeans.centroids, routes))

  0%|          | 0/2039 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

<tf.Variable 'UnreadVariable' shape=(2040, 256) dtype=float32, numpy=
array([[-0.05677956,  0.03678567, -0.09070697, ..., -0.0722867 ,
        -0.00662374, -0.0484607 ],
       [ 0.00336831, -0.02808611, -0.06830411, ..., -0.04615078,
        -0.01346683,  0.0071326 ],
       [ 0.01912913, -0.09788226, -0.03938286, ..., -0.04352076,
         0.04196085,  0.08501685],
       ...,
       [ 0.06827363, -0.07867252,  0.02086764, ..., -0.02036259,
         0.01449005,  0.08939813],
       [-0.04306776,  0.08950221,  0.02987495, ..., -0.05019516,
         0.05872185,  0.08296056],
       [-0.06479619,  0.02037481, -0.09785016, ..., -0.01392203,
         0.02658109, -0.10147958]], dtype=float32)>

In [ ]:
brand_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(name_embeddings)[1], learning_rate=1.)

brand_dict = {k: v for v, k in enumerate(train_df["name"].value_counts().head(255*8).index)}
brand_df = train_df[train_df["name"].isin(brand_dict.keys())].drop_duplicates("name")[["name", "ix"]]
brand_df["brand_ix"] = brand_df["name"].map(brand_dict)
brand_embeddings = tf.scatter_nd(tf.expand_dims(brand_df["brand_ix"].values, axis=1),
                                 tf.gather(name_embeddings, brand_df["ix"].values),
                                 [len(brand_df), tf.shape(name_embeddings)[1]])

data = {}
data["distance_matrix"] = ((1. - tf.einsum("nd,md->nm", brand_embeddings, brand_embeddings)) * 1e9).numpy().astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
brand_df["brand_tsp_ix"] = brand_df["brand_ix"].map({v: k for k, v in enumerate(routes)})

brand_kmeans.centroids.assign(tf.gather(brand_embeddings, routes))


<tf.Variable 'UnreadVariable' shape=(2040, 256) dtype=float32, numpy=
array([[-0.04098925, -0.06080123, -0.02781636, ...,  0.05629943,
         0.00336294,  0.11467186],
       [-0.06531695, -0.05341464, -0.02680026, ...,  0.04961438,
        -0.04938837,  0.10250378],
       [ 0.01846442, -0.06393591,  0.09954432, ...,  0.02021522,
        -0.05345864,  0.02081409],
       ...,
       [-0.08571657, -0.04134927,  0.01134916, ...,  0.07846334,
        -0.06121435,  0.07562376],
       [-0.08205356, -0.02641124, -0.01154182, ...,  0.12519097,
        -0.09823461,  0.08581351],
       [-0.05179709, -0.04558831, -0.03449196, ...,  0.09866905,
        -0.04233998,  0.11391226]], dtype=float32)>

In [ ]:
lrnz

207264

In [ ]:
brand_pid_kmeans = TFMiniBatchSKmeans(255*8, n_dim=tf.shape(name_embeddings)[1], learning_rate=1.)

brand_pid_df = train_df[train_df["name"].isin(brand_dict.keys())][["name", "ix", "pid"]]
brand_pid_df["brand_pid_ix"] = brand_pid_df["name"].map(brand_dict)
not_brand_pid_df = train_df[~train_df["name"].isin(brand_dict.keys())][["ix", "pid"]]
brand_pid_df = brand_pid_df.merge(not_brand_pid_df, on="pid", how="left")
brand_pid_df["ix"] = np.where(brand_pid_df["ix_y"].isna(), brand_pid_df["ix_x"], brand_pid_df["ix_y"]).astype(np.int32)
brand_pid_embeddings = tf.nn.l2_normalize(tf.math.unsorted_segment_mean(tf.gather(name_embeddings, brand_pid_df["ix"].values),
                                                                          brand_pid_df["brand_pid_ix"].values,
                                                                          255*8), axis=-1)



data = {}
data["distance_matrix"] = ((1. - tf.einsum("nd,md->nm", brand_pid_embeddings, brand_pid_embeddings)) * 1e9).numpy().astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
brand_pid_df["brand_pid_tsp_ix"] = brand_pid_df["brand_pid_ix"].map({v: k for k, v in enumerate(routes)})

brand_pid_kmeans.centroids.assign(tf.gather(brand_pid_embeddings, routes))


<tf.Variable 'UnreadVariable' shape=(2040, 256) dtype=float32, numpy=
array([[-0.03828888, -0.03829809, -0.02672131, ...,  0.00162226,
         0.11269024, -0.05158409],
       [-0.02390731, -0.01677149, -0.09876301, ...,  0.01229284,
         0.06156249, -0.08371226],
       [-0.02154939,  0.03645154,  0.04138213, ..., -0.06648063,
         0.03844417, -0.04540368],
       ...,
       [-0.04124658, -0.04311758,  0.03895446, ..., -0.02510447,
         0.13534428, -0.0195477 ],
       [-0.04237897, -0.03922258,  0.04113974, ..., -0.02296809,
         0.13632789, -0.01559151],
       [-0.04126463, -0.04455078,  0.0377099 , ..., -0.02197584,
         0.13566008, -0.01893293]], dtype=float32)>

In [ ]:
hkmeans.save_weights(nadare_feature_dir + "haversine_kmeans")
name_kmeans.save_weights(nadare_feature_dir + "name_kmeans")
address_kmeans.save_weights(nadare_feature_dir + "address_kmeans")
category_kmeans.save_weights(nadare_feature_dir + "category_kmeans")
ix_values_kmeans.save_weights(nadare_feature_dir + "ix_value_kmeans")
mix_kmeans.save_weights(nadare_feature_dir + "mix_kmeans")
brand_kmeans.save_weights(nadare_feature_dir + "brand_kmeans")
brand_pid_kmeans.save_weights(nadare_feature_dir + "brand_pid_kmeans")

category_df.to_csv(nadare_feature_dir + "category_tsp_index.csv", index=None, encoding="utf-8")
brand_df.to_csv(nadare_feature_dir + "brand_tsp.csv", index=None, encoding="utf-8")
brand_pid_df.to_csv(nadare_feature_dir + "brand_pid_tsp.csv", index=None, encoding="utf-8")
classify_model.save_weights(nadare_feature_dir + "classify_model")
mixer_model.save_weights(nadare_feature_dir + "mixer_model")

In [ ]:
city_index_df

,city,city_count,city_index,latitude,longitude
0,Singapore,10259,0,1.327829,103.849073
1,Москва,9643,1,55.722583,37.593779
2,Bandung,8745,2,-6.904392,107.624720
3,New York,8666,3,40.720292,-73.881549
4,İstanbul,6455,4,41.015430,28.983876
...,...,...,...,...,...
2019,Wisconsin Dells,59,2019,43.580736,-89.750238
2020,Hangzhou,59,2020,30.280548,120.187566
2021,Aarschot,59,2021,50.986640,4.821829
2022,Ypsilanti,59,2022,42.246791,-83.633531


In [ ]:
centroid_position = np.deg2rad(city_index_df[["latitude", "longitude"]].values)
data = {}
data["distance_matrix"] = (haversine(tf.expand_dims(centroid_position, axis=1), tf.expand_dims(centroid_position, axis=0)).numpy() * 1e9).astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
city_index_df["city_pos_tsp_index"] = city_index_df["city_index"].map({r: i for i, r in enumerate(routes)})

In [ ]:
centroid_position = np.deg2rad(state_index_df[["latitude", "longitude"]].values)
data = {}
data["distance_matrix"] = (haversine(tf.expand_dims(centroid_position, axis=1), tf.expand_dims(centroid_position, axis=0)).numpy() * 1e9).astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
state_index_df["state_pos_tsp_index"] = state_index_df["state_index"].map({r: i for i, r in enumerate(routes)})

In [ ]:
centroid_position = np.deg2rad(geo_name_index_df[["latitude", "longitude"]].values)
data = {}
data["distance_matrix"] = (haversine(tf.expand_dims(centroid_position, axis=1), tf.expand_dims(centroid_position, axis=0)).numpy() * 1e9).astype(np.int64).tolist()

data["num_vehicles"] = 1
data["depot"] = 0


from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes[0]

manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)


solution = routing.SolveWithParameters(search_parameters)
routes = get_routes(solution, routing, manager)[:-1]
routes = [routes[(i+len(routes)//2)%len(routes)] for i in range(len(routes))]
geo_name_index_df["geo_name_pos_tsp_index"] = geo_name_index_df["geo_name_index"].map({r: i for i, r in enumerate(routes)})

In [ ]:
city_index_df.to_csv(nadare_feature_dir + "city_tsp_index.csv", index=None, encoding="utf-8")
state_index_df.to_csv(nadare_feature_dir + "state_tsp_index.csv", index=None, encoding="utf-8")
geo_name_index_df.to_csv(nadare_feature_dir + "geo_name_tsp_index.csv", index=None, encoding="utf-8")


In [ ]:
country_index_df.to_csv(nadare_feature_dir + "country_tsp_index.csv", index=None, encoding="utf-8")